In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ****Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, Subset
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import efficientnet_b0
from PIL import Image
import random
import torchvision.transforms.functional as TF
from tqdm.notebook import tqdm
import cv2
import zipfile
from collections import Counter
import shutil
import os
import shutil
from pathlib import Path
from torchvision.utils import save_image
from PIL import Image
import time
import copy
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from sklearn.metrics import confusion_matrix, roc_curve, auc, classification_report
from sklearn.preprocessing import label_binarize
from sklearn.metrics import accuracy_score
import time
from datetime import datetime
from torch.optim.lr_scheduler import ReduceLROnPlateau
from itertools import cycle
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, Subset
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import efficientnet_b0
import torchvision.transforms.functional as TF
from torchvision.models import ViT_B_16_Weights, vit_b_16
from torchvision.models import vit_b_16, ViT_B_16_Weights
from collections import OrderedDict
import copy
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import cv2
from torchvision import transforms, models
from datetime import datetime
from collections import Counter
from torch.utils.data import DataLoader, WeightedRandomSampler
import torchvision.utils as vutils
import torch.nn.functional as F
from torch.optim.lr_scheduler import OneCycleLR
from sklearn.preprocessing import label_binarize
from itertools import cycle
import glob

# ****Data Analysis

In [ ]:

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


DATASET_PATH = '/kaggle/input/train-fish/Freshwater Fish Disease Aquaculture in south asia/Train'

class FishDiseaseDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        
        self.samples = []
        for class_name in self.classes:
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.samples.append((os.path.join(class_dir, img_name), self.class_to_idx[class_name], class_name))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label, class_name = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        return {
            'image': image,
            'label': label,
            'path': img_path,
            'class_name': class_name
        }

# analyze image properties
def analyze_image_properties(dataset):
    image_sizes = []
    aspect_ratios = []
    channels = []
    file_sizes = []
    
    for i in tqdm(range(len(dataset)), desc="Analyzing images"):
        sample = dataset[i]
        img_path = sample['path']
        
        
        file_size = os.path.getsize(img_path) / 1024
        file_sizes.append(file_size)
        
        
        with Image.open(img_path) as img:
            width, height = img.size
            image_sizes.append((width, height))
            aspect_ratios.append(width / height)
            channels.append(len(img.getbands()))
    
    return {
        'image_sizes': image_sizes,
        'aspect_ratios': aspect_ratios,
        'channels': channels,
        'file_sizes': file_sizes
    }

# compute pixel statistics
def compute_pixel_statistics(dataset, num_samples=1000):
    # Use a subset of images for pixel statistics to save time
    indices = random.sample(range(len(dataset)), min(num_samples, len(dataset)))
    
    
    to_tensor = transforms.ToTensor()
    
    
    all_pixels = []
    
    for idx in tqdm(indices, desc="Computing pixel statistics"):
        sample = dataset[idx]
        img_path = sample['path']
        
        
        with Image.open(img_path) as img:
            img_tensor = to_tensor(img)
            all_pixels.append(img_tensor.view(3, -1))
    

    all_pixels = torch.cat(all_pixels, dim=1)
    
    
    mean = all_pixels.mean(dim=1)
    std = all_pixels.std(dim=1)
    
    return {
        'mean': mean.tolist(),
        'std': std.tolist()
    }

#  visualize class distribution
def visualize_class_distribution(dataset):
    class_counts = Counter([sample[1] for sample in dataset.samples])
    class_names = [dataset.classes[idx] for idx in sorted(class_counts.keys())]
    counts = [class_counts[idx] for idx in sorted(class_counts.keys())]
    
    plt.figure(figsize=(12, 6))
    bars = plt.bar(class_names, counts, color='skyblue')
    plt.title('Class Distribution in Fish Disease Dataset', fontsize=15)
    plt.xlabel('Disease Classes', fontsize=12)
    plt.ylabel('Number of Images', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    
    
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 5,
                 f'{height}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig('class_distribution.png')
    plt.show()
    
    
    return pd.DataFrame({
        'Class': class_names,
        'Count': counts,
        'Percentage': [count/sum(counts)*100 for count in counts]
    })

# visualize sample images from each class
def visualize_sample_images(dataset, samples_per_class=5):
    # Group samples by class
    class_samples = {}
    for path, label, class_name in dataset.samples:
        if class_name not in class_samples:
            class_samples[class_name] = []
        class_samples[class_name].append(path)
    
    
    selected_samples = {}
    for class_name, paths in class_samples.items():
        selected_samples[class_name] = random.sample(paths, min(samples_per_class, len(paths)))
    
    
    num_classes = len(dataset.classes)
    fig, axes = plt.subplots(num_classes, samples_per_class, figsize=(15, 2.5*num_classes))
    
    for i, class_name in enumerate(dataset.classes):
        for j, img_path in enumerate(selected_samples[class_name]):
            img = Image.open(img_path).convert('RGB')
            axes[i, j].imshow(img)
            axes[i, j].axis('off')
            
            
            if j == 0:
                axes[i, j].set_title(f"{class_name}", fontsize=10)
    
    plt.tight_layout()
    plt.savefig('sample_images.png')
    plt.show()

# analyze image dimensions
def analyze_image_dimensions(image_sizes):
    widths, heights = zip(*image_sizes)
    
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist(widths, bins=20, alpha=0.7, color='blue')
    plt.title('Distribution of Image Widths')
    plt.xlabel('Width (pixels)')
    plt.ylabel('Frequency')
    
    plt.subplot(1, 2, 2)
    plt.hist(heights, bins=20, alpha=0.7, color='green')
    plt.title('Distribution of Image Heights')
    plt.xlabel('Height (pixels)')
    plt.ylabel('Frequency')
    
    plt.tight_layout()
    plt.savefig('image_dimensions.png')
    plt.show()
    
    
    plt.figure(figsize=(10, 8))
    plt.scatter(widths, heights, alpha=0.5)
    plt.title('Image Dimensions: Width vs Height')
    plt.xlabel('Width (pixels)')
    plt.ylabel('Height (pixels)')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig('width_vs_height.png')
    plt.show()
    
    
    return pd.DataFrame({
        'Statistic': ['Min', 'Max', 'Mean', 'Median', 'Std Dev'],
        'Width': [min(widths), max(widths), np.mean(widths), np.median(widths), np.std(widths)],
        'Height': [min(heights), max(heights), np.mean(heights), np.median(heights), np.std(heights)]
    })

# analyze aspect ratios
def analyze_aspect_ratios(aspect_ratios):
    plt.figure(figsize=(10, 6))
    plt.hist(aspect_ratios, bins=30, alpha=0.7, color='purple')
    plt.title('Distribution of Image Aspect Ratios (Width/Height)')
    plt.xlabel('Aspect Ratio')
    plt.ylabel('Frequency')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig('aspect_ratios.png')
    plt.show()
    
    
    return pd.DataFrame({
        'Statistic': ['Min', 'Max', 'Mean', 'Median', 'Std Dev'],
        'Aspect Ratio': [
            min(aspect_ratios),
            max(aspect_ratios),
            np.mean(aspect_ratios),
            np.median(aspect_ratios),
            np.std(aspect_ratios)
        ]
    })

# analyze file sizes
def analyze_file_sizes(file_sizes):
    plt.figure(figsize=(10, 6))
    plt.hist(file_sizes, bins=30, alpha=0.7, color='orange')
    plt.title('Distribution of Image File Sizes')
    plt.xlabel('File Size (KB)')
    plt.ylabel('Frequency')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig('file_sizes.png')
    plt.show()
    
    
    return pd.DataFrame({
        'Statistic': ['Min', 'Max', 'Mean', 'Median', 'Std Dev', 'Total (MB)'],
        'File Size': [
            min(file_sizes),
            max(file_sizes),
            np.mean(file_sizes),
            np.median(file_sizes),
            np.std(file_sizes),
            sum(file_sizes) / 1024  # Total in MB
        ]
    })

# analyze color distributions
def analyze_color_distributions(dataset, num_samples=100):
    # Use a subset of images for color analysis
    indices = random.sample(range(len(dataset)), min(num_samples, len(dataset)))
    
    
    class_color_histograms = {class_name: [] for class_name in dataset.classes}
    
    for idx in tqdm(indices, desc="Analyzing color distributions"):
        sample = dataset[idx]
        img_path = sample['path']
        class_name = sample['class_name']
        
        
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        
        color_hist = []
        for i, color in enumerate(['r', 'g', 'b']):
            hist = cv2.calcHist([img], [i], None, [256], [0, 256])
            hist = hist / hist.sum()  # Normalize
            color_hist.append(hist)
        
        class_color_histograms[class_name].append(color_hist)
    
    
    plt.figure(figsize=(15, 10))
    
    for i, class_name in enumerate(dataset.classes):
        if not class_color_histograms[class_name]:
            continue
            
        plt.subplot(3, 3, i+1)
        
        
        avg_hist = np.zeros((3, 256))
        for hist in class_color_histograms[class_name]:
            for channel in range(3):
                avg_hist[channel] += hist[channel].flatten()
        
        if len(class_color_histograms[class_name]) > 0:
            avg_hist /= len(class_color_histograms[class_name])
        
        # Plot
        colors = ['r', 'g', 'b']
        for channel in range(3):
            plt.plot(avg_hist[channel], color=colors[channel], alpha=0.7)
        
        plt.title(f'Color Distribution: {class_name}')
        plt.xlabel('Pixel Intensity')
        plt.ylabel('Frequency')
        plt.xlim([0, 256])
    
    plt.tight_layout()
    plt.savefig('color_distributions.png')
    plt.show()

#  create a zip file with analyzed images
def create_analysis_zip(dataset, output_zip='analyzed_images.zip', samples_per_class=10):
    # Create a temporary directory to store selected images
    temp_dir = 'temp_images_for_zip'
    os.makedirs(temp_dir, exist_ok=True)
    
    
    class_samples = {}
    for path, label, class_name in dataset.samples:
        if class_name not in class_samples:
            class_samples[class_name] = []
        class_samples[class_name].append(path)
    
    
    selected_samples = {}
    for class_name, paths in class_samples.items():
        selected_samples[class_name] = random.sample(paths, min(samples_per_class, len(paths)))
    
    
    for class_name, paths in selected_samples.items():
        class_dir = os.path.join(temp_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)
        
        for i, img_path in enumerate(paths):
            img_name = f"{i+1}_{os.path.basename(img_path)}"
            shutil.copy(img_path, os.path.join(class_dir, img_name))
    
    
    with zipfile.ZipFile(output_zip, 'w') as zipf:
        for root, dirs, files in os.walk(temp_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, temp_dir)
                zipf.write(file_path, arcname)
    
    
    shutil.rmtree(temp_dir)
    
    print(f"Created zip file '{output_zip}' with sample images")
    return output_zip

# Main 
def analyze_fish_disease_dataset():
    print("Starting Fish Disease Dataset Analysis...")
    
    
    if not os.path.exists(DATASET_PATH):
        print(f"Dataset path {DATASET_PATH} not found. Please update the path.")
        return
    
    
    dataset = FishDiseaseDataset(DATASET_PATH)
    print(f"Found {len(dataset)} images across {len(dataset.classes)} classes")
    print(f"Classes: {dataset.classes}")
    
    # 1. Class Distribution Analysis
    print("\n1. Analyzing class distribution...")
    class_df = visualize_class_distribution(dataset)
    print(class_df)
    
    # 2. Sample Images Visualization
    print("\n2. Visualizing sample images from each class...")
    visualize_sample_images(dataset)
    
    # 3. Image Properties Analysis
    print("\n3. Analyzing image properties...")
    image_props = analyze_image_properties(dataset)
    
    # 4. Image Dimensions Analysis
    print("\n4. Analyzing image dimensions...")
    dim_stats = analyze_image_dimensions(image_props['image_sizes'])
    print(dim_stats)
    
    # 5. Aspect Ratio Analysis
    print("\n5. Analyzing aspect ratios...")
    ar_stats = analyze_aspect_ratios(image_props['aspect_ratios'])
    print(ar_stats)
    
    # 6. File Size Analysis
    print("\n6. Analyzing file sizes...")
    size_stats = analyze_file_sizes(image_props['file_sizes'])
    print(size_stats)
    
    # 7. Color Distribution Analysis
    print("\n7. Analyzing color distributions...")
    analyze_color_distributions(dataset)
    
    # 8. Pixel Statistics
    print("\n8. Computing pixel statistics...")
    pixel_stats = compute_pixel_statistics(dataset)
    print(f"Channel means: {pixel_stats['mean']}")
    print(f"Channel standard deviations: {pixel_stats['std']}")
    
    # 9. Create zip file with analyzed images
    print("\n9. Creating zip file with sample images...")
    zip_path = create_analysis_zip(dataset)
    
    print("\nAnalysis complete! All visualizations have been saved as PNG files.")
    print(f"Sample images have been saved to {zip_path}")
    
    # Return a summary of findings
    return {
        'num_images': len(dataset),
        'classes': dataset.classes,
        'class_distribution': class_df,
        'dimension_stats': dim_stats,
        'aspect_ratio_stats': ar_stats,
        'file_size_stats': size_stats,
        'pixel_stats': pixel_stats
    }


if __name__ == "__main__":
    results = analyze_fish_disease_dataset()

# ****Preprocessing

In [ ]:

def preprocess_dataset(input_dir, output_dir, target_size=(224, 224)):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    
    class_folders = [f for f in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, f))]
    print(f"Found {len(class_folders)} classes: {class_folders}")
    
    
    total_images = 0
    corrupted_images = 0
    
    
    for class_name in class_folders:
        class_path = os.path.join(input_dir, class_name)
        output_class_path = os.path.join(output_dir, class_name)
        
        
        os.makedirs(output_class_path, exist_ok=True)
        
        
        image_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
        print(f"Processing {len(image_files)} images in class '{class_name}'")
        
        
        for img_file in tqdm(image_files, desc=f"Class {class_name}"):
            total_images += 1
            img_path = os.path.join(class_path, img_file)
            output_img_path = os.path.join(output_class_path, img_file)
            
            try:
                
                with Image.open(img_path) as img:
                    
                    resized_img = img.resize(target_size, Image.LANCZOS)
                    
                    
                    resized_img.save(output_img_path)
            except Exception as e:
                print(f"Corrupted or invalid image: {img_path}")
                print(f"Error: {str(e)}")
                corrupted_images += 1
    
    print("\nPreprocessing completed!")
    print(f"Total images processed: {total_images}")
    print(f"Corrupted images found: {corrupted_images}")
    print(f"Preprocessed dataset saved to: {output_dir}")


if __name__ == "__main__":
    
    input_dataset_path = "/kaggle/input/train-fish/Freshwater Fish Disease Aquaculture in south asia/Train"
    output_dataset_path = "/kaggle/working/preprocessed_fish_disease_dataset"
    
    
    preprocess_dataset(input_dataset_path, output_dataset_path, target_size=(224, 224))

# ****Data Split

In [ ]:

random.seed(42)


original_data_dir = '/kaggle/working/preprocessed_fish_disease_dataset'
base_output_dir = '/kaggle/working/'


splits = ['train', 'val', 'test']
for split in splits:
    for class_name in os.listdir(original_data_dir):
        split_path = os.path.join(base_output_dir, split, class_name)
        os.makedirs(split_path, exist_ok=True)


for class_name in os.listdir(original_data_dir):
    class_path = os.path.join(original_data_dir, class_name)
    images = os.listdir(class_path)
    random.shuffle(images)
    
    total_images = len(images)
    train_end = int(0.8 * total_images)
    val_end = int(0.9 * total_images)
    
    train_images = images[:train_end]
    val_images = images[train_end:val_end]
    test_images = images[val_end:]
    
    for image in train_images:
        src = os.path.join(class_path, image)
        dst = os.path.join(base_output_dir, 'train', class_name, image)
        shutil.copy2(src, dst)
        
    for image in val_images:
        src = os.path.join(class_path, image)
        dst = os.path.join(base_output_dir, 'val', class_name, image)
        shutil.copy2(src, dst)
        
    for image in test_images:
        src = os.path.join(class_path, image)
        dst = os.path.join(base_output_dir, 'test', class_name, image)
        shutil.copy2(src, dst)

print("Dataset split completed!")

# ****Augmenting

In [ ]:

train_dir = '/kaggle/working/train'


before_counts = {}
for class_folder in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_folder)
    if os.path.isdir(class_path):
        count = len(os.listdir(class_path))
        before_counts[class_folder] = count

print("Image count BEFORE augmentation:")
for class_name, count in before_counts.items():
    print(f"{class_name}: {count} images")

In [ ]:

augment_transform = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=(0.7, 1.3)),
    transforms.RandomAffine(degrees=0, shear=10),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ToTensor()
])

 
train_dir = '/kaggle/working/train'


augmentations_per_image = 1  


for class_folder in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_folder)
    if os.path.isdir(class_path):
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            try:
                img = Image.open(img_path).convert('RGB')  # Load image safely

                for i in range(augmentations_per_image):
                    augmented = augment_transform(img)
                    save_path = os.path.join(class_path, f"aug_{random.randint(10000,99999)}.jpg")
                    save_image(augmented, save_path)
            
            except Exception as e:
                print(f"Error processing {img_path}: {e}")

In [ ]:

train_dir = '/kaggle/working/train'


before_counts = {}
for class_folder in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_folder)
    if os.path.isdir(class_path):
        count = len(os.listdir(class_path))
        before_counts[class_folder] = count

print("Image count AFTER augmentation:")
for class_name, count in before_counts.items():
    print(f"{class_name}: {count} images")

# ****EfficientNet

In [ ]:

torch.manual_seed(42)
np.random.seed(42)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


TRAIN_PATH = "/kaggle/working/train"
VAL_PATH = "/kaggle/working/val"
TEST_PATH = "/kaggle/working/test"
OUTPUT_DIR = "/kaggle/working/"


os.makedirs(OUTPUT_DIR, exist_ok=True)


augment_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])


val_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])


train_dataset = ImageFolder(TRAIN_PATH, transform=augment_transform)
val_dataset = ImageFolder(VAL_PATH, transform=val_test_transform)
test_dataset = ImageFolder(TEST_PATH, transform=val_test_transform)


batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


class_names = train_dataset.classes
num_classes = len(class_names)
print(f"Classes: {class_names}")
print(f"Number of classes: {num_classes}")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")


def load_model(model_name='efficientnet_b0', num_classes=7):
    if model_name == 'efficientnet_b0':
        model = models.efficientnet_b0(weights='IMAGENET1K_V1')
    elif model_name == 'efficientnet_b1':
        model = models.efficientnet_b1(weights='IMAGENET1K_V1')
    elif model_name == 'efficientnet_b2':
        model = models.efficientnet_b2(weights='IMAGENET1K_V1')
    elif model_name == 'efficientnet_b3':
        model = models.efficientnet_b3(weights='IMAGENET1K_V1')
    else:
        model = models.efficientnet_b0(weights='IMAGENET1K_V1')
    
    
    for param in list(model.parameters())[:-20]:  # Freeze all but the last few layers
        param.requires_grad = False
    
    
    in_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(in_features=in_features, out_features=num_classes)
    )
    
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'Trainable parameters: {trainable_params:,}')
    return model

# train the model
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, early_stopping_patience=10):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    
    no_improve_epochs = 0
    
    
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
                dataloader = train_loader
            else:
                model.eval()   
                dataloader = val_loader
            
            running_loss = 0.0
            running_corrects = 0
            
            
            for inputs, labels in tqdm(dataloader, desc=phase):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                
                optimizer.zero_grad()
                
                # Forward pass
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # Backward + optimize only in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            
            if phase == 'train':
                train_losses.append(epoch_loss)
                train_accs.append(epoch_acc.item())
            else:
                val_losses.append(epoch_loss)
                val_accs.append(epoch_acc.item())
                
                
                scheduler.step(epoch_loss)
                
                
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    no_improve_epochs = 0
                else:
                    no_improve_epochs += 1
        
        
        if no_improve_epochs >= early_stopping_patience:
            print(f'Early stopping triggered after {epoch+1} epochs')
            break
            
        print()
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')
    
    
    model.load_state_dict(best_model_wts)
    
    
    return model, {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accs': train_accs,
        'val_accs': val_accs
    }

# evaluate model on test set
def evaluate_model(model, dataloader):
    model.eval()
    
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc='Testing'):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    
    accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    print(f'Test Accuracy: {accuracy:.4f}')
    
    
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))
    
    return all_preds, all_labels, all_probs


def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'confusion_matrix.png'), dpi=300, bbox_inches='tight')
    plt.show()


def plot_learning_curves(history):
    plt.figure(figsize=(12, 5))
    
    
    plt.subplot(1, 2, 1)
    plt.plot(history['train_losses'], label='Training Loss')
    plt.plot(history['val_losses'], label='Validation Loss')
    plt.title('Loss Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    
    
    plt.subplot(1, 2, 2)
    plt.plot(history['train_accs'], label='Training Accuracy')
    plt.plot(history['val_accs'], label='Validation Accuracy')
    plt.title('Accuracy Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'learning_curves.png'), dpi=300, bbox_inches='tight')
    plt.show()


def plot_roc_curves(y_true, y_probs, classes):
    
    y_true_bin = label_binarize(y_true, classes=range(len(classes)))
    
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range(len(classes)):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    
    plt.figure(figsize=(12, 10))
    
    for i in range(len(classes)):
        plt.plot(fpr[i], tpr[i], lw=2, label=f'{classes[i]} (AUC = {roc_auc[i]:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves for Each Class')
    plt.legend(loc="lower right")
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'roc_curves.png'), dpi=300, bbox_inches='tight')
    plt.show()


def check_and_fix_fitting(history):
    train_losses = history['train_losses']
    val_losses = history['val_losses']
    train_accs = history['train_accs']
    val_accs = history['val_accs']
    
    
    loss_gap = np.array(train_losses) - np.array(val_losses)
    acc_gap = np.array(train_accs) - np.array(val_accs)
    
    
    if np.mean(loss_gap[-5:]) < -0.1 and np.mean(acc_gap[-5:]) > 0.1:
        print("Detected OVERFITTING: Training accuracy is much higher than validation accuracy.")
        return "overfitting"
    
    
    elif np.mean(train_accs[-5:]) < 0.7 and np.mean(val_accs[-5:]) < 0.7:
        print("Detected UNDERFITTING: Both training and validation accuracy are low.")
        return "underfitting"
    
    
    else:
        print("Model appears to be well-fitted: No significant overfitting or underfitting detected.")
        return "good_fit"


def apply_fitting_strategies(diagnosis, model, optimizer):
    if diagnosis == "overfitting":
        print("Applying strategies to reduce overfitting...")
        
        #  Increase dropout
        for module in model.modules():
            if isinstance(module, nn.Dropout):
                module.p = min(module.p + 0.1, 0.5)  # Increase dropout but cap at 0.5
                print(f"Increased dropout to {module.p}")
        
        # Apply weight decay
        for param_group in optimizer.param_groups:
            param_group['weight_decay'] = 1e-4
            print(f"Added weight decay: {param_group['weight_decay']}")
        
        return model
        
    elif diagnosis == "underfitting":
        print("Applying strategies to reduce underfitting...")
        
        # Decrease dropout
        for module in model.modules():
            if isinstance(module, nn.Dropout):
                module.p = max(module.p - 0.1, 0.1)  # Decrease dropout but keep at least 0.1
                print(f"Decreased dropout to {module.p}")
        
        # Unfreeze more layers for fine-tuning
        ct = 0
        for child in model.children():
            ct += 1
            if ct > 6:  # Unfreeze more layers
                for param in child.parameters():
                    param.requires_grad = True
        
        print("Unfrozen more layers for fine-tuning")
        
        # Increase learning rate
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 2
            print(f"Increased learning rate to {param_group['lr']}")
        
        return model
    
    else:  
        print("Model is well-fitted. No changes needed.")
        return model


def main():
    
    model = load_model(model_name='efficientnet_b0', num_classes=num_classes)
    model = model.to(device)
    
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
    
    print("Starting training...")
    
    model, history = train_model(
        model, 
        criterion, 
        optimizer, 
        scheduler,
        num_epochs=20,
        early_stopping_patience=7
    )
    
    
    plot_learning_curves(history)
    
    
    diagnosis = check_and_fix_fitting(history)
    
    
    if diagnosis != "good_fit":
        print("\nApplying strategies and retraining...")
        model = apply_fitting_strategies(diagnosis, model, optimizer)
        
        
        optimizer = optim.Adam(model.parameters(), lr=0.0005)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
        
        
        model, history2 = train_model(
            model, 
            criterion, 
            optimizer, 
            scheduler,
            num_epochs=15,
            early_stopping_patience=5
        )
        
        
        plot_learning_curves(history2)
    
    
    torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, 'efficientnet_fish_disease.pth'))
    print(f"Model saved to {os.path.join(OUTPUT_DIR, 'efficientnet_fish_disease.pth')}")
    
    
    print("\nEvaluating on test set...")
    y_pred, y_true, y_probs = evaluate_model(model, test_loader)
    
    
    plot_confusion_matrix(y_true, y_pred, class_names)
    
    
    plot_roc_curves(y_true, y_probs, class_names)
    
    print("\nAll plots have been saved to the output directory.")

if __name__ == "__main__":
    main()

# ****DenseNet

In [ ]:

imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]
augment_transform = transforms.Compose([
    
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
    
])



data_transform = {
    'train': augment_transform,
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
    ])
}


data_dir = "/kaggle/working/"
datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), transform=data_transform[x]) for x in ['train', 'val', 'test']}
loaders = {x: DataLoader(datasets[x], batch_size=32, shuffle=True) for x in ['train', 'val', 'test']}
class_names = datasets['train'].classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pretrained DenseNet model
model = models.densenet121(pretrained=True)

# Replace the classifier for the current task
num_features = model.classifier.in_features
model.classifier = nn.Linear(num_features, len(class_names))
model = model.to(device)


def train_and_validate(model, loaders, criterion, optimizer, num_epochs=10):
    train_loss, val_loss = [], []
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        
        # Training phase
        model.train()
        running_loss = 0.0
        for inputs, labels in loaders['train']:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_loss.append(running_loss / len(loaders['train']))

        # Validation phase
        model.eval()
        val_running_loss = 0.0
        with torch.no_grad():
            for inputs, labels in loaders['val']:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()
        val_loss.append(val_running_loss / len(loaders['val']))

        print(f"Train Loss: {train_loss[-1]:.4f}, Val Loss: {val_loss[-1]:.4f}")
    return train_loss, val_loss


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_loss, val_loss = train_and_validate(model, loaders, criterion, optimizer,num_epochs=20)

plt.figure()
plt.plot(train_loss, label="Train Loss")  # Training Loss
plt.plot(val_loss, label="Validation Loss")  # Validation Loss
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Learning Curve")
plt.savefig("learning_curve_densenet.png")  # Save the learning curve image
plt.show()


if val_loss[-1] > train_loss[-1] and (val_loss[-1] - train_loss[-1]) > 0.05:
    print("The DenseNet model is overfitting.")
elif val_loss[-1] < train_loss[-1] and (train_loss[-1] - val_loss[-1]) > 0.05:
    print("The DenseNet model is underfitting.")
else:
    print("The DenseNet model is fitting well.")


plt.figure()
plt.plot(train_loss, label="Train Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Learning Curve")
plt.savefig("learning_curve.png")
plt.show()


def test_model(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_labels, all_preds

labels, preds = test_model(model, loaders['test'])


report = classification_report(labels, preds, target_names=class_names)
print("Classification Report:")
print(report)


test_accuracy = accuracy_score(labels, preds)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


cm = confusion_matrix(labels, preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.savefig("confusion_matrix.png")
plt.show()



def evaluate_model_probs(model, loader):
    model.eval()
    all_probs, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probabilities = torch.softmax(outputs, dim=1)  # Get probabilities for all classes
            all_probs.extend(probabilities.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_labels), np.array(all_probs)


labels, outputs_probs = evaluate_model_probs(model, loaders['test'])


from sklearn.preprocessing import label_binarize
labels_one_hot = label_binarize(labels, classes=range(len(class_names)))


fpr, tpr, roc_auc = {}, {}, {}
for i in range(len(class_names)):
    fpr[i], tpr[i], _ = roc_curve(labels_one_hot[:, i], outputs_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])


plt.figure(figsize=(10, 8))
for i in range(len(class_names)):
    plt.plot(fpr[i], tpr[i], label=f"Class {class_names[i]} (AUC = {roc_auc[i]:.2f})")
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line for random guessing
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC AUC Curve")
plt.legend(loc="lower right")
plt.savefig("roc_auc_curve.png")
plt.show()




   


****Overfit

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

augment_transform = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=(0.7, 1.3)),
    transforms.RandomAffine(degrees=0, shear=10),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
])

data_transform = {
    'train': augment_transform,
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
    ])
}


data_dir = "/kaggle/working/"  
datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), transform=data_transform[x]) for x in ['train', 'val', 'test']}
loaders = {x: DataLoader(datasets[x], batch_size=32, shuffle=True) for x in ['train', 'val', 'test']}
class_names = datasets['train'].classes


model = models.densenet121(pretrained=True)


for param in model.features.parameters():
    param.requires_grad = False  

for name, param in list(model.features.named_parameters())[-5:]:  # Unfreeze the last 5 layers for fine-tuning
    param.requires_grad = True

num_features = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Dropout(p=0.3),  # Reduced dropout rate
    nn.Linear(num_features, len(class_names))
)
model = model.to(device)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Trainable parameters: {trainable_params:,}')



criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)  


def train_and_validate(model, loaders, criterion, optimizer, scheduler, num_epochs=30, patience=5):
    train_loss, val_loss = [], []
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        
        model.train()
        running_loss = 0.0
        for inputs, labels in loaders['train']:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_loss.append(running_loss / len(loaders['train']))

        
        model.eval()
        val_running_loss = 0.0
        with torch.no_grad():
            for inputs, labels in loaders['val']:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()
        val_loss.append(val_running_loss / len(loaders['val']))

        print(f"Train Loss: {train_loss[-1]:.4f}, Val Loss: {val_loss[-1]:.4f}")

        
        scheduler.step()

        
        if val_loss[-1] < best_val_loss:
            best_val_loss = val_loss[-1]
            patience_counter = 0
            torch.save(model.state_dict(), "best_model.pth")  # Save best model
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

    
    print("\nFit Assessment:")
    if val_loss[-1] > train_loss[-1] and (val_loss[-1] - train_loss[-1]) > 0.05:
        print("The DenseNet model is overfitting.")
    elif val_loss[-1] < train_loss[-1] and (train_loss[-1] - val_loss[-1]) > 0.05:
        print("The DenseNet model is underfitting.")
    else:
        print("The DenseNet model is fitting well.")

    return train_loss, val_loss


train_loss, val_loss = train_and_validate(model, loaders, criterion, optimizer, scheduler)


plt.figure()
plt.plot(train_loss, label="Train Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Learning Curve")
plt.savefig("learning_curve_densenet.png")
plt.show()


def evaluate_model(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_labels, all_preds

labels, preds = evaluate_model(model, loaders['test'])


report = classification_report(labels, preds, target_names=class_names)
print("\nClassification Report:")
print(report)


test_accuracy = accuracy_score(labels, preds)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")


cm = confusion_matrix(labels, preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.savefig("confusion_matrix_densenet.png")
plt.show()


def evaluate_model_probs(model, loader):
    model.eval()
    all_probs, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probabilities = torch.softmax(outputs, dim=1)
            all_probs.extend(probabilities.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_labels), np.array(all_probs)

labels, outputs_probs = evaluate_model_probs(model, loaders['test'])
labels_one_hot = label_binarize(labels, classes=range(len(class_names)))

fpr, tpr, roc_auc = {}, {}, {}
for i in range(len(class_names)):
    fpr[i], tpr[i], _ = roc_curve(labels_one_hot[:, i], outputs_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(10, 8))
for i in range(len(class_names)):
    plt.plot(fpr[i], tpr[i], label=f"Class {class_names[i]} (AUC = {roc_auc[i]:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC AUC Curve")
plt.legend(loc="lower right")
plt.savefig("roc_auc_curve_densenet.png")
plt.show()


# ****ResNet50

In [ ]:

imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

augment_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
])

data_transform = {
    'train': augment_transform,
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
    ])
}


data_dir = "/kaggle/working/"  
datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), transform=data_transform[x]) for x in ['train', 'val', 'test']}
loaders = {x: DataLoader(datasets[x], batch_size=32, shuffle=True) for x in ['train', 'val', 'test']}
class_names = datasets['train'].classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained ResNet50 model
model = models.resnet50(pretrained=True)

# Replace the final layer to fit the dataset
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(class_names))
model = model.to(device)

def train_and_validate(model, loaders, criterion, optimizer, num_epochs=20):
    train_loss, val_loss = [], []
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        
        # Training phase
        model.train()
        running_loss = 0.0
        for inputs, labels in loaders['train']:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_loss.append(running_loss / len(loaders['train']))

        # Validation phase
        model.eval()
        val_running_loss = 0.0
        with torch.no_grad():
            for inputs, labels in loaders['val']:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()
        val_loss.append(val_running_loss / len(loaders['val']))

        print(f"Train Loss: {train_loss[-1]:.4f}, Val Loss: {val_loss[-1]:.4f}")
    return train_loss, val_loss

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_loss, val_loss = train_and_validate(model, loaders, criterion, optimizer)

plt.figure()
plt.plot(train_loss, label="Train Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Learning Curve")
plt.savefig("learning_curve.png")
plt.show()


if val_loss[-1] > train_loss[-1] and val_loss[-1] - train_loss[-1] > 0.05:
    print("The model is overfitting.")
elif val_loss[-1] > train_loss[-1]:
    print("The model is slightly overfitting.")
elif train_loss[-1] > val_loss[-1]:
    print("The model is underfitting.")
else:
    print("The model is fitting well.")

def evaluate_model(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_labels, all_preds

labels, preds = evaluate_model(model, loaders['test'])

cm = confusion_matrix(labels, preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.savefig("confusion_matrix.png")
plt.show()



def evaluate_model_probs(model, loader):
    model.eval()
    all_probs, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probabilities = torch.softmax(outputs, dim=1)
            all_probs.extend(probabilities.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_labels), np.array(all_probs)

labels, outputs_probs = evaluate_model_probs(model, loaders['test'])
labels_one_hot = label_binarize(labels, classes=range(len(class_names)))

fpr, tpr, roc_auc = {}, {}, {}
for i in range(len(class_names)):
    fpr[i], tpr[i], _ = roc_curve(labels_one_hot[:, i], outputs_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(10, 8))
for i in range(len(class_names)):
    plt.plot(fpr[i], tpr[i], label=f"Class {class_names[i]} (AUC = {roc_auc[i]:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC AUC Curve")
plt.legend(loc="lower right")
plt.savefig("roc_auc_curve.png")
plt.show()






Overfit

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

augment_transform = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=(0.7, 1.3)),
    transforms.RandomAffine(degrees=0, shear=10),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5),  # Advanced augmentation
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
])

data_transform = {
    'train': augment_transform,
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
    ])
}


data_dir = "/kaggle/working/"  
datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), transform=data_transform[x]) for x in ['train', 'val', 'test']}
loaders = {x: DataLoader(datasets[x], batch_size=32, shuffle=True) for x in ['train', 'val', 'test']}
class_names = datasets['train'].classes

from torchvision import models

# Replace ResNet50 with ResNet34 for reduced model complexity
model = models.resnet34(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(p=0.5),  
    nn.Linear(num_features, len(class_names))
)
model = model.to(device)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Trainable parameters: {trainable_params:,}')



def train_and_validate(model, loaders, criterion, optimizer, num_epochs=20, patience=5):
    train_loss, val_loss = [], []
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        # Training phase
        model.train()
        running_loss = 0.0
        for inputs, labels in loaders['train']:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_loss.append(running_loss / len(loaders['train']))

        # Validation phase
        model.eval()
        val_running_loss = 0.0
        with torch.no_grad():
            for inputs, labels in loaders['val']:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()
        val_loss.append(val_running_loss / len(loaders['val']))

        print(f"Train Loss: {train_loss[-1]:.4f}, Val Loss: {val_loss[-1]:.4f}")

        
        if val_loss[-1] < best_val_loss:
            best_val_loss = val_loss[-1]
            patience_counter = 0
            torch.save(model.state_dict(), "best_model.pth")  # Save best model
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

    
    print("\nFit Assessment:")
    if val_loss[-1] > train_loss[-1] and (val_loss[-1] - train_loss[-1]) > 0.05:
        print("The ResNet34 model is overfitting.")
    elif val_loss[-1] < train_loss[-1] and (train_loss[-1] - val_loss[-1]) > 0.05:
        print("The ResNet34 model is underfitting.")
    else:
        print("The ResNet34 model is fitting well.")

    return train_loss, val_loss


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)


train_loss, val_loss = train_and_validate(model, loaders, criterion, optimizer)


plt.figure()
plt.plot(train_loss, label="Train Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Learning Curve")
plt.savefig("learning_curve.png")
plt.show()


def evaluate_model(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_labels, all_preds

labels, preds = evaluate_model(model, loaders['test'])


report = classification_report(labels, preds, target_names=class_names)
print("\nClassification Report:")
print(report)


test_accuracy = accuracy_score(labels, preds)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")

# Confusion matri
cm = confusion_matrix(labels, preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.savefig("confusion_matrix.png")
plt.show()


def evaluate_model_probs(model, loader):
    model.eval()
    all_probs, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probabilities = torch.softmax(outputs, dim=1)
            all_probs.extend(probabilities.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_labels), np.array(all_probs)

labels, outputs_probs = evaluate_model_probs(model, loaders['test'])
labels_one_hot = label_binarize(labels, classes=range(len(class_names)))

fpr, tpr, roc_auc = {}, {}, {}
for i in range(len(class_names)):
    fpr[i], tpr[i], _ = roc_curve(labels_one_hot[:, i], outputs_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(10, 8))
for i in range(len(class_names)):
    plt.plot(fpr[i], tpr[i], label=f"Class {class_names[i]} (AUC = {roc_auc[i]:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC AUC Curve")
plt.legend(loc="lower right")
plt.savefig("roc_auc_curve.png")
plt.show()


# ****AquaScan

In [ ]:

torch.manual_seed(42)
np.random.seed(42)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}


data_dir = '/kaggle/working/'  
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) 
                 for x in ['train', 'val', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
              for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes
num_classes = len(class_names)

print(f"Classes: {class_names}")
print(f"Number of training images: {dataset_sizes['train']}")
print(f"Number of validation images: {dataset_sizes['val']}")
print(f"Number of test images: {dataset_sizes['test']}")

# Define the AquaScan CNN model
class AquaScanCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(AquaScanCNN, self).__init__()
        
        # Convolutional Block 1
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25)
        )
        
        # Convolutional Block 2
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25)
        )
        
        # Convolutional Block 3
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3)
        )
        
        # Convolutional Block 4
        self.conv_block4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.4)
        )
        
        
        self.fc_input_size = 14 * 14 * 256
        
        # Fully Connected Block
        self.fc_block = nn.Sequential(
            nn.Linear(self.fc_input_size, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5)
        )
        
        # Output Layer
        self.output_layer = nn.Linear(256, num_classes)
        
    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc_block(x)
        x = self.output_layer(x)
        
        return x


model = AquaScanCNN(num_classes=num_classes).to(device)
print(model)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)


def train_model(model, criterion, optimizer, scheduler, num_epochs=50, early_stopping_patience=7):
    since = time.time()
    
   
    best_model_wts = model.state_dict()
    best_acc = 0.0
    
    
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    
    
    best_val_loss = float('inf')
    early_stopping_counter = 0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   
                
            running_loss = 0.0
            running_corrects = 0
            
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                
                optimizer.zero_grad()
                
                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            
            if phase == 'train':
                train_losses.append(epoch_loss)
                train_accs.append(epoch_acc.item())
            else:
                val_losses.append(epoch_loss)
                val_accs.append(epoch_acc.item())
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            
            if phase == 'val':
                
                scheduler.step(epoch_loss)
                
                
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = model.state_dict().copy()
                
                
                if epoch_loss < best_val_loss:
                    best_val_loss = epoch_loss
                    early_stopping_counter = 0
                else:
                    early_stopping_counter += 1
                    if early_stopping_counter >= early_stopping_patience:
                        print(f'Early stopping triggered after {epoch+1} epochs')
                        
                        model.load_state_dict(best_model_wts)
                        
                        
                        plot_learning_curves(train_losses, val_losses, train_accs, val_accs)
                        
                        
                        check_fitting(train_losses, val_losses)
                        
                        time_elapsed = time.time() - since
                        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
                        print(f'Best val Acc: {best_acc:4f}')
                        
                        return model, train_losses, val_losses, train_accs, val_accs
        
        print()
    
    
    model.load_state_dict(best_model_wts)
    
    
    plot_learning_curves(train_losses, val_losses, train_accs, val_accs)
    
    
    check_fitting(train_losses, val_losses)
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')
    
    return model, train_losses, val_losses, train_accs, val_accs


def check_fitting(train_losses, val_losses):
    
    final_train_loss = train_losses[-1]
    final_val_loss = val_losses[-1]
    loss_gap = final_val_loss - final_train_loss
    
    print("\n--- Fitting Analysis ---")
    
    
    if final_train_loss > 0.3:  
        print("Model may be UNDERFITTING:")
       
    
    
    elif loss_gap > 0.1:  
        print("Model may be OVERFITTING:")
       
    
    else:
        print("Model seems to be fitting well!")
        


def plot_learning_curves(train_losses, val_losses, train_accs, val_accs):
    plt.figure(figsize=(12, 5))
    
    
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Loss Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    
    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Training Accuracy')
    plt.plot(val_accs, label='Validation Accuracy')
    plt.title('Accuracy Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plt.savefig(f'learning_curves_{timestamp}.png')
    plt.show()


def evaluate_model(model, test_loader):
    model.eval()
    
    
    all_preds = []
    all_labels = []
    running_corrects = 0
    
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            
            running_corrects += torch.sum(preds == labels.data)
    
    
    test_acc = running_corrects.double() / dataset_sizes['test']
    print(f'Test Accuracy: {test_acc:.4f}')
    
    return all_preds, all_labels, test_acc.item()


def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plt.savefig(f'confusion_matrix_{timestamp}.png')
    plt.show()


def plot_roc_curve(y_test, y_score, classes):
    # Binarize the labels for multi-class ROC
    y_test_bin = label_binarize(y_test, classes=range(len(classes)))
    n_classes = y_test_bin.shape[1]
    
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    
    plt.figure(figsize=(10, 8))
    colors = cycle(['blue', 'red', 'green', 'yellow', 'purple', 'orange', 'brown'])
    
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label=f'ROC curve of class {classes[i]} (area = {roc_auc[i]:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Multi-class ROC Curve')
    plt.legend(loc="lower right")
    
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plt.savefig(f'roc_curve_{timestamp}.png')
    plt.show()


def generate_classification_report(y_true, y_pred, classes):
    report = classification_report(y_true, y_pred, target_names=classes, output_dict=True)
    
    
    import pandas as pd
    df_report = pd.DataFrame(report).transpose()
    
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(df_report.iloc[:-3, :3], annot=True, cmap='Blues', fmt='.2f')
    plt.title('Classification Report')
    
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plt.savefig(f'classification_report_{timestamp}.png')
    plt.show()
    
    
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=classes))


if __name__ == "__main__":
    
    print("Starting model training...")
    model, train_losses, val_losses, train_accs, val_accs = train_model(
        model, criterion, optimizer, scheduler, num_epochs=50, early_stopping_patience=7
    )
    
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    torch.save(model.state_dict(), f'aquascan_model_{timestamp}.pth')
    print(f"Model saved as aquascan_model_{timestamp}.pth")
    
    
    print("\nEvaluating model on test set...")
    all_preds, all_labels, test_acc = evaluate_model(model, dataloaders['test'])
    
    
    print("\nGenerating confusion matrix...")
    plot_confusion_matrix(all_labels, all_preds, class_names)
    
    
    print("\nGenerating ROC curve...")
    
    model.eval()
    y_score = []
    with torch.no_grad():
        for inputs, _ in dataloaders['test']:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            y_score.extend(probs.cpu().numpy())
    
    plot_roc_curve(all_labels, np.array(y_score), class_names)
    
    
    print("\nGenerating classification report...")
    generate_classification_report(all_labels, all_preds, class_names)
    
    print("\nAll analysis complete!")


****Hybrid

In [ ]:

torch.manual_seed(42)
np.random.seed(42)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


data_dir = '/kaggle/working/'  


def analyze_dataset(data_dir):
    print("\n=== DATASET ANALYSIS ===")
    
    
    class_counts = {}
    for split in ['train', 'val', 'test']:
        split_dir = os.path.join(data_dir, split)
        if not os.path.exists(split_dir):
            print(f"WARNING: {split_dir} directory does not exist!")
            continue
            
        
        split_class_counts = {}
        for class_name in os.listdir(split_dir):
            class_dir = os.path.join(split_dir, class_name)
            if os.path.isdir(class_dir):
                num_images = len([f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))])
                split_class_counts[class_name] = num_images
        
        print(f"\n{split.upper()} set:")
        for class_name, count in split_class_counts.items():
            print(f"  - {class_name}: {count} images")
        
        total = sum(split_class_counts.values())
        print(f"  Total: {total} images")
        
        
        if split_class_counts:
            min_count = min(split_class_counts.values())
            max_count = max(split_class_counts.values())
            imbalance_ratio = max_count / min_count if min_count > 0 else float('inf')
            print(f"  Class imbalance ratio (max/min): {imbalance_ratio:.2f}")
            if imbalance_ratio > 2:
                print("  WARNING: Significant class imbalance detected!")
        
        if split == 'train':
            class_counts = split_class_counts
    
    print("\n=== END OF DATASET ANALYSIS ===\n")
    return class_counts


class_counts = analyze_dataset(data_dir)


data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}


try:
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) 
                    for x in ['train', 'val', 'test']}
    
    
    if class_counts:
        class_weights = [1.0 / class_counts[image_datasets['train'].classes[i]] 
                        for i in range(len(image_datasets['train'].classes))]
        sample_weights = [class_weights[label] for _, label in image_datasets['train'].samples]
        sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)
        
        
        dataloaders = {
            'train': DataLoader(image_datasets['train'], batch_size=16, sampler=sampler, num_workers=4),
            'val': DataLoader(image_datasets['val'], batch_size=16, shuffle=False, num_workers=4),
            'test': DataLoader(image_datasets['test'], batch_size=16, shuffle=False, num_workers=4)
        }
    else:
        dataloaders = {x: DataLoader(image_datasets[x], batch_size=16, shuffle=(x == 'train'), num_workers=4)
                    for x in ['train', 'val', 'test']}
    
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
    class_names = image_datasets['train'].classes
    num_classes = len(class_names)
    
    print(f"Classes: {class_names}")
    print(f"Number of training images: {dataset_sizes['train']}")
    print(f"Number of validation images: {dataset_sizes['val']}")
    print(f"Number of test images: {dataset_sizes['test']}")
    
    
    def visualize_batch(dataloader, title):
        batch = next(iter(dataloader))
        images, labels = batch
        
        
        grid = vutils.make_grid(images[:16], nrow=4, normalize=True)
        plt.figure(figsize=(10, 10))
        plt.title(title)
        plt.imshow(grid.permute(1, 2, 0).cpu())
        plt.axis('off')
        plt.show()
        
        
        print("Labels for visualized images:")
        for i, label in enumerate(labels[:16]):
            print(f"Image {i+1}: {class_names[label]}")
    
    print("\nVisualizing training images:")
    visualize_batch(dataloaders['train'], "Training Images")
    
except Exception as e:
    print(f"Error loading datasets: {e}")
    raise

# Define the AquaScan-EfficientNet hybrid model
class AquaScanEfficientNetHybrid(nn.Module):
    def __init__(self, num_classes=7, model_name='efficientnet_b0', use_pretrained=True):
        super(AquaScanEfficientNetHybrid, self).__init__()
        
        # Load pre-trained EfficientNet
        if model_name == 'efficientnet_b0':
            self.backbone = models.efficientnet_b0(weights='IMAGENET1K_V1' if use_pretrained else None)
        elif model_name == 'efficientnet_b1':
            self.backbone = models.efficientnet_b1(weights='IMAGENET1K_V1' if use_pretrained else None)
        elif model_name == 'efficientnet_b2':
            self.backbone = models.efficientnet_b2(weights='IMAGENET1K_V1' if use_pretrained else None)
        else:
            raise ValueError(f"Unsupported model name: {model_name}")
        
        
        self.features = nn.Sequential(*list(self.backbone.children())[:-1])
        
        
        if model_name == 'efficientnet_b0':
            backbone_features = 1280
        elif model_name == 'efficientnet_b1':
            backbone_features = 1280
        elif model_name == 'efficientnet_b2':
            backbone_features = 1408
        
        
        self.activation = nn.LeakyReLU(0.1, inplace=True)
        
        # Original AquaScan convolutional blocks with residual connections
        # Block 1 with residual connection
        self.conv_block1_main = nn.Sequential(
            nn.Conv2d(backbone_features, backbone_features, kernel_size=3, padding=1),
            nn.BatchNorm2d(backbone_features),
            self.activation,
            nn.Conv2d(backbone_features, backbone_features, kernel_size=3, padding=1),
            nn.BatchNorm2d(backbone_features),
            nn.Dropout(0.1)
        )
        
        # Block 2 with residual connection
        self.conv_block2_main = nn.Sequential(
            nn.Conv2d(backbone_features, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            self.activation,
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.Dropout(0.1)
        )
        self.conv_block2_shortcut = nn.Sequential(
            nn.Conv2d(backbone_features, 512, kernel_size=1, stride=1),
            nn.BatchNorm2d(512)
        )
        
        # Global Average Pooling
        self.gap = nn.AdaptiveAvgPool2d(1)
        
        # Fully Connected Blocks with more capacity
        self.fc_block = nn.Sequential(
            nn.Linear(512, 1024),
            self.activation,
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            self.activation,
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            self.activation,
            nn.Dropout(0.3)
        )
        
        # Output Layer
        self.output_layer = nn.Linear(256, num_classes)
        
        # Initially freeze the backbone
        self.freeze_backbone()
        
    def freeze_backbone(self):
        for param in self.features.parameters():
            param.requires_grad = False
            
    def unfreeze_backbone(self):
        for param in self.features.parameters():
            param.requires_grad = True
    
    def unfreeze_layers(self, num_layers):
        # Unfreeze the last num_layers of the backbone
        children = list(self.features.children())
        for child in children[-num_layers:]:
            for param in child.parameters():
                param.requires_grad = True
    
    def forward(self, x):
        # Get features from EfficientNet backbone
        x = self.features(x)
        
        # Block 1 with residual connection
        identity = x
        x = self.conv_block1_main(x)
        x = x + identity  # Residual connection
        x = self.activation(x)
        
        # Block 2 with residual connection
        identity = self.conv_block2_shortcut(x)
        x = self.conv_block2_main(x)
        x = x + identity  # Residual connection
        x = self.activation(x)
        
        # Global Average Pooling
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = self.fc_block(x)
        x = self.output_layer(x)
        
        return x


model = AquaScanEfficientNetHybrid(num_classes=num_classes, model_name='efficientnet_b0').to(device)
print(model)


def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

total_params = count_trainable_parameters(model)
print(f"Total Trainable Parameters: {total_params}")


criterion = nn.CrossEntropyLoss(label_smoothing=0.1)


optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), 
                       lr=0.001, weight_decay=1e-4)


steps_per_epoch = len(dataloaders['train'])
total_steps = steps_per_epoch * 100  # 100 epochs max


scheduler = OneCycleLR(
    optimizer,
    max_lr=0.01,
    total_steps=total_steps,
    pct_start=0.1,
    div_factor=25,
    final_div_factor=1000
)


def train_model(model, criterion, optimizer, scheduler, num_epochs=100, early_stopping_patience=10):
    since = time.time()
    
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    
    
    best_val_loss = float('inf')
    early_stopping_counter = 0
    
    
    unfreeze_schedule = {
        10: 2,   
        20: 4,   
        30: 6    
    }
    
    # Training loop
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        
        if epoch in unfreeze_schedule:
            print(f"Unfreezing the last {unfreeze_schedule[epoch]} layers of the backbone")
            model.unfreeze_layers(unfreeze_schedule[epoch])
            
            
            optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), 
                                  lr=0.0005, weight_decay=1e-4)
            
            
            remaining_epochs = num_epochs - epoch
            total_steps = steps_per_epoch * remaining_epochs
            
            
            scheduler = OneCycleLR(
                optimizer,
                max_lr=0.001,  
                total_steps=total_steps,
                pct_start=0.1,
                div_factor=25,
                final_div_factor=1000
            )
        
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   
                
            running_loss = 0.0
            running_corrects = 0
            all_preds = []
            all_labels = []
            
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                
                optimizer.zero_grad()
                
                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        # Gradient clipping
                        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                        optimizer.step()
                        scheduler.step()
                
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
            
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            
            per_class_acc = {}
            for i, class_name in enumerate(class_names):
                class_mask = np.array(all_labels) == i
                if np.sum(class_mask) > 0:  # Avoid division by zero
                    class_correct = np.sum((np.array(all_preds) == i) & class_mask)
                    per_class_acc[class_name] = class_correct / np.sum(class_mask)
                else:
                    per_class_acc[class_name] = 0.0
            
            
            if phase == 'train':
                train_losses.append(epoch_loss)
                train_accs.append(epoch_acc.item())
            else:
                val_losses.append(epoch_loss)
                val_accs.append(epoch_acc.item())
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            
            print("Per-class accuracy:")
            for class_name, acc in per_class_acc.items():
                print(f"  {class_name}: {acc:.4f}")
            
            
            if phase == 'val':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    
                    early_stopping_counter = 0
                
                
                if epoch_loss < best_val_loss:
                    best_val_loss = epoch_loss
                else:
                    early_stopping_counter += 1
                    if early_stopping_counter >= early_stopping_patience:
                        print(f'Early stopping triggered after {epoch+1} epochs')
                        
                        model.load_state_dict(best_model_wts)
                        
                        
                        plot_learning_curves(train_losses, val_losses, train_accs, val_accs)
                        
                        time_elapsed = time.time() - since
                        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
                        print(f'Best val Acc: {best_acc:4f}')
                        
                        return model, train_losses, val_losses, train_accs, val_accs
        
        
        for param_group in optimizer.param_groups:
            print(f'Current learning rate: {param_group["lr"]:.6f}')
        
        print()
    
    
    model.load_state_dict(best_model_wts)
    
    
    plot_learning_curves(train_losses, val_losses, train_accs, val_accs)
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')
    
    return model, train_losses, val_losses, train_accs, val_accs


def plot_learning_curves(train_losses, val_losses, train_accs, val_accs):
    plt.figure(figsize=(12, 5))
    
    
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Loss Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    
    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Training Accuracy')
    plt.plot(val_accs, label='Validation Accuracy')
    plt.title('Accuracy Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plt.savefig(f'learning_curves_{timestamp}.png')
    plt.show()


def evaluate_model(model, test_loader):
    model.eval()
    
    
    all_preds = []
    all_labels = []
    all_probs = []
    running_corrects = 0
    
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            probs = F.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
            
            
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            
            running_corrects += torch.sum(preds == labels.data)
    
    
    test_acc = running_corrects.double() / dataset_sizes['test']
    print(f'Test Accuracy: {test_acc:.4f}')
    
    
    per_class_acc = {}
    for i, class_name in enumerate(class_names):
        class_mask = np.array(all_labels) == i
        if np.sum(class_mask) > 0:  # Avoid division by zero
            class_correct = np.sum((np.array(all_preds) == i) & class_mask)
            per_class_acc[class_name] = class_correct / np.sum(class_mask)
        else:
            per_class_acc[class_name] = 0.0
    
    print("Per-class test accuracy:")
    for class_name, acc in per_class_acc.items():
        print(f"  {class_name}: {acc:.4f}")
    
    return all_preds, all_probs, all_labels, test_acc.item()


def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plt.savefig(f'confusion_matrix_{timestamp}.png')
    plt.show()


def plot_roc_curve(y_test, y_score, classes):
    
    y_test_bin = label_binarize(y_test, classes=range(len(classes)))
    n_classes = y_test_bin.shape[1]
    
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    
    plt.figure(figsize=(10, 8))
    colors = cycle(['blue', 'red', 'green', 'yellow', 'purple', 'orange', 'brown'])
    
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label=f'ROC curve of class {classes[i]} (area = {roc_auc[i]:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Multi-class ROC Curve')
    plt.legend(loc="lower right")
    
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plt.savefig(f'roc_curve_{timestamp}.png')
    plt.show()


def generate_classification_report(y_true, y_pred, classes):
    report = classification_report(y_true, y_pred, target_names=classes, output_dict=True)
    
    
    import pandas as pd
    df_report = pd.DataFrame(report).transpose()
    
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(df_report.iloc[:-3, :3], annot=True, cmap='Blues', fmt='.2f')
    plt.title('Classification Report')
    
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plt.savefig(f'classification_report_{timestamp}.png')
    plt.show()
    
    
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=classes))


def test_time_augmentation(model, inputs, num_augmentations=5):
    model.eval()
    batch_size = inputs.size(0)
    
    
    with torch.no_grad():
        outputs = model(inputs)
        probs = F.softmax(outputs, dim=1)
    
    
    all_probs = probs.clone()
    
    
    tta_transforms = [
        transforms.RandomHorizontalFlip(p=1.0),
        transforms.RandomVerticalFlip(p=1.0),
        transforms.RandomRotation(degrees=(90, 90)),
        transforms.RandomRotation(degrees=(180, 180)),
        transforms.RandomRotation(degrees=(270, 270)),
    ]
    
    
    for transform in tta_transforms[:num_augmentations]:
        
        aug_inputs = torch.stack([transform(img) for img in inputs])
        
        
        with torch.no_grad():
            aug_outputs = model(aug_inputs)
            aug_probs = F.softmax(aug_outputs, dim=1)
        
        
        all_probs += aug_probs
    
    
    avg_probs = all_probs / (num_augmentations + 1)
    _, preds = torch.max(avg_probs, 1)
    
    return preds, avg_probs


if __name__ == "__main__":
    
    print("Starting model training...")
    model, train_losses, val_losses, train_accs, val_accs = train_model(
        model, criterion, optimizer, scheduler, num_epochs=100, early_stopping_patience=10
    )
    
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    torch.save(model.state_dict(), f'aquascan_model_{timestamp}.pth')
    print(f"Model saved as aquascan_model_{timestamp}.pth")
    
    
    print("\nEvaluating model on test set ")
    
    
    all_preds = []
    all_probs = []
    all_labels = []
    running_corrects = 0
    
    
    model.eval()
    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        
        preds, probs = test_time_augmentation(model, inputs)
        
        
        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        
        
        running_corrects += torch.sum(preds == labels.data)
    
    
    test_acc = running_corrects.double() / dataset_sizes['test']
    print(f'Test Accuracy with TTA: {test_acc:.4f}')
    
    
    print("\nGenerating confusion matrix...")
    plot_confusion_matrix(all_labels, all_preds, class_names)
    
    
    print("\nGenerating ROC curve...")
    plot_roc_curve(all_labels, np.array(all_probs), class_names)
    
    
    print("\nGenerating classification report...")
    generate_classification_report(all_labels, all_preds, class_names)
    
   

# ****Vision Transformer

In [ ]:

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

#Checking underfit overfit
def diagnose_model_fit(history):
    
    # Get final metrics
    final_train_acc = history['train_acc'][-1]
    final_val_acc = history['val_acc'][-1]
    final_train_loss = history['train_loss'][-1]
    final_val_loss = history['val_loss'][-1]
    
    # Calculate gaps
    acc_gap = final_train_acc - final_val_acc
    loss_gap = final_val_loss - final_train_loss
    
    # Analyze training curves
    train_acc_trend = np.array(history['train_acc'])
    val_acc_trend = np.array(history['val_acc'])
    
    # Check if training accuracy is still improving in the last few epochs
    train_still_improving = train_acc_trend[-1] > np.mean(train_acc_trend[-4:-1]) if len(train_acc_trend) >= 4 else True
    
    # Print diagnostics
    print("\\n===== MODEL FIT DIAGNOSIS =====")
    print(f"Final Training Accuracy: {final_train_acc:.4f}")
    print(f"Final Validation Accuracy: {final_val_acc:.4f}")
    print(f"Accuracy Gap (Train-Val): {acc_gap:.4f}")
    print(f"Loss Gap (Val-Train): {loss_gap:.4f}")
    
    # Determine fit status
    if final_train_acc < 0.9 and train_still_improving:
        print("\\nDIAGNOSIS: UNDERFITTING")
    
    elif acc_gap > 0.1 or loss_gap > 0.1:
        print("\\nDIAGNOSIS: OVERFITTING")
       
    else:
        print("\\nDIAGNOSIS: WELL-FITTED")
        
    
    # Plot learning curves with fit regions
    plt.figure(figsize=(12, 5))
    
    # Plot accuracy with regions
    plt.subplot(1, 2, 1)
    epochs = range(1, len(history['train_acc']) + 1)
    plt.plot(epochs, history['train_acc'], 'b-', label='Training Acc')
    plt.plot(epochs, history['val_acc'], 'r-', label='Validation Acc')
    
    # Add colored regions to indicate fitting status
    plt.fill_between(epochs, history['train_acc'], history['val_acc'], 
                    where=(np.array(history['train_acc']) > np.array(history['val_acc']) + 0.1),
                    color='red', alpha=0.3, label='Overfitting Region')
    plt.fill_between(epochs, [0.9] * len(epochs), [1.0] * len(epochs),
                    where=(np.array(history['train_acc']) < 0.9),
                    color='blue', alpha=0.3, label='Underfitting Region')
    
    plt.title('Model Accuracy with Fit Diagnosis')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_loss'], 'b-', label='Training Loss')
    plt.plot(epochs, history['val_loss'], 'r-', label='Validation Loss')
    
    # Add colored regions to indicate fitting status
    plt.fill_between(epochs, history['train_loss'], history['val_loss'],
                    where=(np.array(history['val_loss']) > np.array(history['train_loss']) + 0.1),
                    color='red', alpha=0.3, label='Overfitting Region')
    
    plt.title('Model Loss with Fit Diagnosis')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'diagnosis': 'underfitting' if final_train_acc < 0.9 and train_still_improving else 
                    'overfitting' if acc_gap > 0.1 or loss_gap > 0.1 else 'well-fitted',
        'metrics': {
            'final_train_acc': final_train_acc,
            'final_val_acc': final_val_acc,
            'acc_gap': acc_gap,
            'loss_gap': loss_gap
        }
    }



# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        
        self.root_dir = root_dir
        self.transform = transform

        # Check if directory exists
        if not os.path.exists(root_dir):
            raise FileNotFoundError(f"Directory not found: {root_dir}")
            
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        
        self.samples = []
        for class_name in self.classes:
            class_dir = os.path.join(root_dir, class_name)
            if not os.path.isdir(class_dir):
                continue
                
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.samples.append((
                        os.path.join(class_dir, img_name),
                        self.class_to_idx[class_name]
                    ))
        
        print(f"Loaded {len(self.samples)} images from {len(self.classes)} classes")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        # Load image
        img = Image.open(img_path).convert('RGB')
        
        # Apply transformations
        if self.transform:
            img = self.transform(img)
            
        return img, label

# Create data transformations with light augmentation
def get_transforms():
   
    train_transform = transforms.Compose([
        transforms.Resize((256, 256)),  
        transforms.RandomCrop(224),     
        transforms.RandomHorizontalFlip(p=0.3),  
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),  
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.1, scale=(0.02, 0.1))  
    ])
    
    # Validation transform (no augmentation)
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return train_transform, val_transform


def create_vit_model(num_classes=7, fine_tune=True, dropout_rate=0.2):
    # Load pre-trained ViT model with ImageNet weights
    weights = ViT_B_16_Weights.DEFAULT
    model = vit_b_16(weights=weights)
    
    print("Vision Transformer architecture:")
    print(model)
    print("\\nUsing transfer learning with ImageNet pre-trained weights")
    
   
    num_features = model.heads.head.in_features
    
   
    model.heads = nn.Sequential(
        nn.LayerNorm(num_features),
        nn.Dropout(dropout_rate), 
        nn.Linear(num_features, 512),  
        nn.GELU(),  
        nn.LayerNorm(512),  
        nn.Dropout(dropout_rate),  
        nn.Linear(512, num_classes)  
    )
    
    # Freeze early layers if fine_tune is True
    if fine_tune:
        # Freeze all parameters
        for param in model.parameters():
            param.requires_grad = False
        
        # Unfreeze the classifier head
        for param in model.heads.parameters():
            param.requires_grad = True
        
        # Unfreeze the last transformer block
        for param in model.encoder.layers[-1].parameters():
            param.requires_grad = True
        
        # Unfreeze layer norm parameters throughout the model
        
        for name, param in model.named_parameters():
            if 'norm' in name:
                param.requires_grad = True
    
    return model

# Training function with mixup data augmentation and gradient clipping
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25, patience=7, 
                mixup_alpha=0.2, clip_grad_norm=1.0):
    since = time.time()
    
    # Initialize variables
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0
    counter = 0  # For early stopping
    
    # For tracking metrics
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }
    
    # Mixup function for data augmentation
    def mixup_data(x, y, alpha=0.2):
        '''Returns mixed inputs, pairs of targets, and lambda'''
        if alpha > 0:
            lam = np.random.beta(alpha, alpha)
        else:
            lam = 1

        batch_size = x.size()[0]
        index = torch.randperm(batch_size).to(device)

        mixed_x = lam * x + (1 - lam) * x[index, :]
        y_a, y_b = y, y[index]
        return mixed_x, y_a, y_b, lam

    def mixup_criterion(criterion, pred, y_a, y_b, lam):
        return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   
                
            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data
            for inputs, labels in tqdm(dataloaders[phase], desc=f"{phase}"):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Zero the parameter gradients
                optimizer.zero_grad()
                
                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    # Apply mixup in training phase
                    if phase == 'train' and mixup_alpha > 0:
                        inputs, labels_a, labels_b, lam = mixup_data(inputs, labels, mixup_alpha)
                        outputs = model(inputs)
                        loss = mixup_criterion(criterion, outputs, labels_a, labels_b, lam)
                        _, preds = torch.max(outputs, 1)
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        _, preds = torch.max(outputs, 1)
                    
                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        
                        # Gradient clipping to prevent exploding gradients
                        if clip_grad_norm > 0:
                            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad_norm)
                            
                        optimizer.step()
                
                # Statistics
                running_loss += loss.item() * inputs.size(0)
                if phase == 'train' and mixup_alpha > 0:
                    # For mixup, we approximate corrects
                    running_corrects += (lam * torch.sum(preds == labels_a.data) + 
                                        (1 - lam) * torch.sum(preds == labels_b.data))
                else:
                    running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            # Store history
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                best_epoch = epoch
                counter = 0  
            elif phase == 'val':
                counter += 1
                if counter >= patience:
                    print(f"Early stopping triggered after {patience} epochs without improvement.")
                    model.load_state_dict(best_model_wts)
                    time_elapsed = time.time() - since
                    print(f"Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
                    print(f"Best val Acc: {best_acc:.4f} at epoch {best_epoch+1}")
                    return model, history
        
        # Step the scheduler if in train phase
        if scheduler is not None:
            scheduler.step()

    # Load best model weights
    model.load_state_dict(best_model_wts)
    
    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
    print(f"Best val Acc: {best_acc:.4f} at epoch {best_epoch+1}")
    
    return model, history

# Evaluate the model
def evaluate_model(model, test_loader, class_names):
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate accuracy
    accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    print(f"Test Accuracy: {accuracy:.4f}")
    
    # Create confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    
    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()
    
    # Print classification report
    print("\\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))
    
    return accuracy, cm

# Plot training history
def plot_training_history(history):
    plt.figure(figsize=(12, 5))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history['train_acc'], label='Train')
    plt.plot(history['val_acc'], label='Validation')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history['train_loss'], label='Train')
    plt.plot(history['val_loss'], label='Validation')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

# Main execution
def main():
    # Kaggle dataset path 
    data_dir = '/kaggle/input/fish-split/Split'  
    
    train_dir = os.path.join(data_dir, 'train')
    val_dir = os.path.join(data_dir, 'val')
    test_dir = os.path.join(data_dir, 'test')
    
    print(f"Data directory: {data_dir}")
    print(f"Data directory exists: {os.path.exists(data_dir)}")
    print(f"Train directory: {train_dir}")
    print(f"Train directory exists: {os.path.exists(train_dir)}")
    
    # Check if directories exist
    for dir_path in [train_dir, val_dir, test_dir]:
        if not os.path.exists(dir_path):
            print(f"Warning: Directory not found: {dir_path}")
    
    # Get transforms with light augmentation to help prevent overfitting
    train_transform, val_transform = get_transforms()
    
    # Create datasets
    train_dataset = CustomDataset(train_dir, transform=train_transform)
    val_dataset = CustomDataset(val_dir, transform=val_transform)
    test_dataset = CustomDataset(test_dir, transform=val_transform)
    
    # Get class names
    class_names = train_dataset.classes
    print(f"Classes: {class_names}")
    
    # Create dataloaders with a smaller batch size to increase variability
    batch_size = 16  
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    
    # Combine into a dictionary for the training function
    dataloaders = {
        'train': train_loader,
        'val': val_loader
    }
    
    # Create model with dropout for regularization
    model = create_vit_model(num_classes=len(class_names), fine_tune=True, dropout_rate=0.2)
    model = model.to(device)
    
    # Set up loss function
    criterion = nn.CrossEntropyLoss()
    
    # Set up optimizer with increased weight decay for regularization
    optimizer = optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()), 
        lr=0.0001, 
        weight_decay=0.05  
    )
    
    # Set up learning rate scheduler 
    from torch.optim.lr_scheduler import OneCycleLR
    
    # Calculate total steps for the scheduler
    total_steps = len(train_loader) * 30  
    
    scheduler = OneCycleLR(
        optimizer,
        max_lr=0.001,
        total_steps=total_steps,
        pct_start=0.1,  
        div_factor=25,  
        final_div_factor=1000,  
        anneal_strategy='cos'  
    )
    
    # Train the model with mixup and gradient clipping
    model, history = train_model(
        model=model,
        dataloaders=dataloaders,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        num_epochs=30,
        patience=10,  
        mixup_alpha=0.2,  
        clip_grad_norm=1.0  
    )
    
    # Plot training history
    plot_training_history(history)
    
    # Check for overfitting/underfitting
    fit_diagnosis = diagnose_model_fit(history)
    print(f"\\nModel fit diagnosis: {fit_diagnosis['diagnosis']}")
    
    # Evaluate the model
    accuracy, cm = evaluate_model(model, test_loader, class_names)
    
    # Save the model
    torch.save(model.state_dict(), 'vit_fish_disease_model_improved.pth')
    print("Model saved")

# Call the main function
if __name__ == "__main__":
    main()




****VIT Parameter Checking

In [ ]:

# Function to create the Vision Transformer model 
def create_vit_model(num_classes=7, fine_tune=True, dropout_rate=0.2):
    # Load pre-trained ViT model with ImageNet weights
    weights = ViT_B_16_Weights.DEFAULT
    model = vit_b_16(weights=weights)
    
   
    num_features = model.heads.head.in_features
    
    
    model.heads = nn.Sequential(
        nn.LayerNorm(num_features),
        nn.Dropout(dropout_rate),  
        nn.Linear(num_features, 512),  
        nn.GELU(),  
        nn.LayerNorm(512),  
        nn.Dropout(dropout_rate),  
        nn.Linear(512, num_classes)  
    )
    
    # Freeze early layers if fine_tune is True
    if fine_tune:
        # Freeze all parameters
        for param in model.parameters():
            param.requires_grad = False
        
        # Unfreeze the classifier head
        for param in model.heads.parameters():
            param.requires_grad = True
        
        # Unfreeze the last transformer block
        for param in model.encoder.layers[-1].parameters():
            param.requires_grad = True
        
        # Unfreeze layer norm parameters throughout the model
        
        for name, param in model.named_parameters():
            if 'norm' in name:
                param.requires_grad = True
    
    return model

# Function to analyze model parameters
def analyze_model_parameters(model, save_dir=None):
    
    print("===== MODEL PARAMETER ANALYSIS =====")
    
    
    if save_dir and not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # 1. Basic parameter statistics
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    frozen_params = total_params - trainable_params
    
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%})")
    print(f"Frozen parameters: {frozen_params:,} ({frozen_params/total_params:.2%})")
    
    # 2. Parameter distribution by layer
    param_data = []
    for name, param in model.named_parameters():
        # Get module name 
        module_name = name.split('.')[0]
        if len(name.split('.')) > 1:
            submodule = name.split('.')[1]
            if submodule.isdigit():  # For indexed layers like encoder.layers.0
                module_name = f"{module_name}.{submodule}"
        
        param_data.append({
            'name': name,
            'module': module_name,
            'shape': tuple(param.shape),
            'size': param.numel(),
            'trainable': param.requires_grad,
            'min': float(param.min()),
            'max': float(param.max()),
            'mean': float(param.mean()),
            'std': float(param.std()),
            'zeros': float((param == 0).sum() / param.numel()),
            'type': param.dtype
        })
    
    
    param_df = pd.DataFrame(param_data)
    
    # 3. Parameters by module
    module_stats = param_df.groupby('module').agg({
        'size': 'sum',
        'trainable': lambda x: sum(x) / len(x) * 100  
    }).sort_values('size', ascending=False)
    
    module_stats['size_percent'] = module_stats['size'] / total_params * 100
    
    print("\nParameters by module:")
    print(module_stats)
    
    # 4. Visualize parameter distribution by module
    plt.figure(figsize=(12, 6))
    
    # Plot parameter count by module
    ax1 = plt.subplot(1, 2, 1)
    bars = ax1.bar(module_stats.index, module_stats['size'], color='skyblue')
    ax1.set_title('Parameter Count by Module')
    ax1.set_ylabel('Number of Parameters')
    ax1.set_xticklabels(module_stats.index, rotation=45, ha='right')
    
    
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:,.0f}',
                ha='center', va='bottom', rotation=0)
    
    
    ax2 = plt.subplot(1, 2, 2)
    bars = ax2.bar(module_stats.index, module_stats['trainable'], color='lightgreen')
    ax2.set_title('Trainable Parameters by Module (%)')
    ax2.set_ylabel('Trainable Parameters (%)')
    ax2.set_xticklabels(module_stats.index, rotation=45, ha='right')
    ax2.set_ylim(0, 100)
    
   
    for bar in bars:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%',
                ha='center', va='bottom', rotation=0)
    
    plt.tight_layout()
    if save_dir:
        plt.savefig(os.path.join(save_dir, 'parameter_distribution.png'), dpi=300, bbox_inches='tight')
    plt.show()
    
    # 5. Analyze parameter statistics
    
    numeric_cols = ['size', 'min', 'max', 'mean', 'std', 'zeros']
    numeric_df = param_df[numeric_cols]
    
    # Correlation matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title('Parameter Statistics Correlation')
    if save_dir:
        plt.savefig(os.path.join(save_dir, 'parameter_correlation.png'), dpi=300, bbox_inches='tight')
    plt.show()
    
    # 6. Parameter value distribution
    plt.figure(figsize=(15, 10))
    
    
    sample_layers = param_df.sample(min(10, len(param_df)))
    
    for i, (_, row) in enumerate(sample_layers.iterrows()):
        param_name = row['name']
        param_tensor = None
        
        
        for name, param in model.named_parameters():
            if name == param_name:
                param_tensor = param.detach().cpu().flatten().numpy()
                break
        
        if param_tensor is not None:
            plt.subplot(3, 4, i+1)
            sns.histplot(param_tensor, kde=True, bins=50)
            plt.title(f"{param_name[:20]}..." if len(param_name) > 20 else param_name)
            plt.xlabel('Value')
            plt.ylabel('Count')
    
    plt.tight_layout()
    if save_dir:
        plt.savefig(os.path.join(save_dir, 'parameter_distributions.png'), dpi=300, bbox_inches='tight')
    plt.show()
    
    # 7. Check for potential issues
    print("\nPotential issues:")
    
    
    has_nan = any(torch.isnan(p).any() for p in model.parameters())
    has_inf = any(torch.isinf(p).any() for p in model.parameters())
    print(f"- NaN values detected: {has_nan}")
    print(f"- Inf values detected: {has_inf}")
    
    
    dead_params = param_df[param_df['zeros'] > 0.99]
    if not dead_params.empty:
        print(f"- Potential dead parameters (>99% zeros): {len(dead_params)}")
        print(dead_params[['name', 'zeros', 'size']])
    else:
        print("- No dead parameters detected")
    
    
    extreme_params = param_df[(param_df['max'] > 100) | (param_df['min'] < -100)]
    if not extreme_params.empty:
        print(f"- Parameters with extreme values (>100 or <-100): {len(extreme_params)}")
        print(extreme_params[['name', 'min', 'max', 'mean', 'std']])
    else:
        print("- No extreme parameter values detected")
    
    return {
        'total_params': total_params,
        'trainable_params': trainable_params,
        'frozen_params': frozen_params,
        'param_df': param_df,
        'module_stats': module_stats
    }

# Function to analyze model architecture
def analyze_model_architecture(model):
    """
    Analyze the architecture of a PyTorch model.
    """
    print("\n===== MODEL ARCHITECTURE ANALYSIS =====")
    
    
    print(model)
    
    
    layer_counts = {}
    for name, module in model.named_modules():
        layer_type = module.__class__.__name__
        if layer_type in layer_counts:
            layer_counts[layer_type] += 1
        else:
            layer_counts[layer_type] = 1
    
    
    for container in ['Sequential', 'ModuleList', 'ModuleDict']:
        if container in layer_counts:
            del layer_counts[container]
    
    
    layer_counts = OrderedDict(sorted(layer_counts.items(), key=lambda x: x[1], reverse=True))
    
    print("\nLayer types:")
    for layer_type, count in layer_counts.items():
        print(f"- {layer_type}: {count}")
    
    
    plt.figure(figsize=(12, 6))
    plt.bar(layer_counts.keys(), layer_counts.values(), color='lightblue')
    plt.title('Model Layer Composition')
    plt.xlabel('Layer Type')
    plt.ylabel('Count')
    plt.xticks(rotation=45, ha='right')
    
    
    for i, (layer_type, count) in enumerate(layer_counts.items()):
        plt.text(i, count, str(count), ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    return layer_counts

# Function to analyze model weights from a saved model file
def analyze_saved_model(model_path, num_classes=7):
    """
    Analyze a saved model's parameters.
    
    Args:
        model_path: Path to the saved model file
        num_classes: Number of classes in the model
    """
    print(f"\n===== ANALYZING SAVED MODEL: {model_path} =====")
    
    
    model = create_vit_model(num_classes=num_classes)
    
    
    try:
        model.load_state_dict(torch.load(model_path, map_location='cpu'))
        print("Model loaded successfully!")
    except Exception as e:
        print(f"Error loading model: {e}")
        return None
    
    
    model.eval()
    
    
    arch_analysis = analyze_model_architecture(model)
    param_analysis = analyze_model_parameters(model)
    
    return model, arch_analysis, param_analysis

# Main execution
if __name__ == "__main__":
    
    print("Creating a new Vision Transformer model...")
    model = create_vit_model(num_classes=7)
    
    
    arch_analysis = analyze_model_architecture(model)
    
    
    param_analysis = analyze_model_parameters(model, save_dir='model_analysis')
    
   

# ****VGG 16

In [ ]:


torch.manual_seed(42)
np.random.seed(42)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


def diagnose_model_fit(history):
    
    final_train_acc = history['train_acc'][-1]
    final_val_acc = history['val_acc'][-1]
    final_train_loss = history['train_loss'][-1]
    final_val_loss = history['val_loss'][-1]
    
    
    acc_gap = final_train_acc - final_val_acc
    loss_gap = final_val_loss - final_train_loss
    
    
    train_acc_trend = np.array(history['train_acc'])
    val_acc_trend = np.array(history['val_acc'])
    
    
    train_still_improving = train_acc_trend[-1] > np.mean(train_acc_trend[-4:-1]) if len(train_acc_trend) >= 4 else True
    
    
    print("\n===== MODEL FIT DIAGNOSIS =====")
    print(f"Final Training Accuracy: {final_train_acc:.4f}")
    print(f"Final Validation Accuracy: {final_val_acc:.4f}")
    print(f"Accuracy Gap (Train-Val): {acc_gap:.4f}")
    print(f"Loss Gap (Val-Train): {loss_gap:.4f}")
    
    
    if final_train_acc < 0.9 and train_still_improving:
        print("\nDIAGNOSIS: UNDERFITTING")
    elif acc_gap > 0.1 or loss_gap > 0.1:
        print("\nDIAGNOSIS: OVERFITTING")
    else:
        print("\nDIAGNOSIS: WELL-FITTED")
    
    
    plt.figure(figsize=(12, 5))
    
    
    plt.subplot(1, 2, 1)
    epochs = range(1, len(history['train_acc']) + 1)
    plt.plot(epochs, history['train_acc'], 'b-', label='Training Acc')
    plt.plot(epochs, history['val_acc'], 'r-', label='Validation Acc')
    
    
    plt.fill_between(epochs, history['train_acc'], history['val_acc'], 
                    where=(np.array(history['train_acc']) > np.array(history['val_acc']) + 0.1),
                    color='red', alpha=0.3, label='Overfitting Region')
    plt.fill_between(epochs, [0.9] * len(epochs), [1.0] * len(epochs),
                    where=(np.array(history['train_acc']) < 0.9),
                    color='blue', alpha=0.3, label='Underfitting Region')
    
    plt.title('Model Accuracy with Fit Diagnosis')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_loss'], 'b-', label='Training Loss')
    plt.plot(epochs, history['val_loss'], 'r-', label='Validation Loss')
    
    
    plt.fill_between(epochs, history['train_loss'], history['val_loss'],
                    where=(np.array(history['val_loss']) > np.array(history['train_loss']) + 0.1),
                    color='red', alpha=0.3, label='Overfitting Region')
    
    plt.title('Model Loss with Fit Diagnosis')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'diagnosis': 'underfitting' if final_train_acc < 0.9 and train_still_improving else 
                    'overfitting' if acc_gap > 0.1 or loss_gap > 0.1 else 'well-fitted',
        'metrics': {
            'final_train_acc': final_train_acc,
            'final_val_acc': final_val_acc,
            'acc_gap': acc_gap,
            'loss_gap': loss_gap
        }
    }

# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        
        if not os.path.exists(root_dir):
            raise FileNotFoundError(f"Directory not found: {root_dir}")
            
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        
        self.samples = []
        for class_name in self.classes:
            class_dir = os.path.join(root_dir, class_name)
            if not os.path.isdir(class_dir):
                continue
                
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.samples.append((
                        os.path.join(class_dir, img_name),
                        self.class_to_idx[class_name]
                    ))
        
        print(f"Loaded {len(self.samples)} images from {len(self.classes)} classes")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        
        img = Image.open(img_path).convert('RGB')
        
        
        if self.transform:
            img = self.transform(img)
            
        return img, label


def get_transforms():
   
    train_transform = transforms.Compose([
        transforms.Resize((256, 256)),  
        transforms.RandomCrop(224),    
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.3),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.3, scale=(0.02, 0.2))  
    ])
    
    
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return train_transform, val_transform


class CustomVGG16(nn.Module):
    def __init__(self, num_classes=7, dropout_rate=0.3):
        super(CustomVGG16, self).__init__()
        
        
        vgg16 = models.vgg16(pretrained=True)
        
        
        self.features = vgg16.features
        
        
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x


def create_vgg16_model(num_classes=7, fine_tune=True, dropout_rate=0.3):
   
    model = CustomVGG16(num_classes=num_classes, dropout_rate=dropout_rate)
    
    print("Custom VGG16 architecture created")
    print("\nUsing transfer learning with ImageNet pre-trained weights")
    
    
    if fine_tune:
        for param in model.features[:17].parameters():  
            param.requires_grad = False
        
        
        for param in model.features[17:].parameters():
            param.requires_grad = True
        
        
        for param in model.classifier.parameters():
            param.requires_grad = True
    
    return model


def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25, patience=10, 
                mixup_alpha=0.2, clip_grad_norm=1.0):
    since = time.time()
    
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0
    counter = 0  # For early stopping
    
   
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }
    
    
    def mixup_data(x, y, alpha=0.2):
        
        if alpha > 0:
            lam = np.random.beta(alpha, alpha)
        else:
            lam = 1

        batch_size = x.size()[0]
        index = torch.randperm(batch_size).to(device)

        mixed_x = lam * x + (1 - lam) * x[index, :]
        y_a, y_b = y, y[index]
        return mixed_x, y_a, y_b, lam

    def mixup_criterion(criterion, pred, y_a, y_b, lam):
        return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   
                
            running_loss = 0.0
            running_corrects = 0
            
            
            for inputs, labels in tqdm(dataloaders[phase], desc=f"{phase}"):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                
                optimizer.zero_grad()
                
               
                with torch.set_grad_enabled(phase == 'train'):
                    
                    if phase == 'train' and mixup_alpha > 0:
                        inputs, labels_a, labels_b, lam = mixup_data(inputs, labels, mixup_alpha)
                        outputs = model(inputs)
                        loss = mixup_criterion(criterion, outputs, labels_a, labels_b, lam)
                        _, preds = torch.max(outputs, 1)
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        _, preds = torch.max(outputs, 1)
                    
                    
                    if phase == 'train':
                        loss.backward()
                        
                        
                        if clip_grad_norm > 0:
                            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad_norm)
                            
                        optimizer.step()
                
                
                running_loss += loss.item() * inputs.size(0)
                if phase == 'train' and mixup_alpha > 0:
                    
                    running_corrects += (lam * torch.sum(preds == labels_a.data) + 
                                        (1 - lam) * torch.sum(preds == labels_b.data))
                else:
                    running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                best_epoch = epoch
                counter = 0  
            elif phase == 'val':
                counter += 1
                if counter >= patience:
                    print(f"Early stopping triggered after {patience} epochs without improvement.")
                    model.load_state_dict(best_model_wts)
                    time_elapsed = time.time() - since
                    print(f"Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
                    print(f"Best val Acc: {best_acc:.4f} at epoch {best_epoch+1}")
                    return model, history
        
        
        if scheduler is not None:
            scheduler.step()

    
    model.load_state_dict(best_model_wts)
    
    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
    print(f"Best val Acc: {best_acc:.4f} at epoch {best_epoch+1}")
    
    return model, history


def evaluate_model(model, test_loader, class_names):
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    
    accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    print(f"Test Accuracy: {accuracy:.4f}")
    
    
    cm = confusion_matrix(all_labels, all_preds)
    
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()
    
    
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))
    
    return accuracy, cm


def plot_training_history(history):
    plt.figure(figsize=(12, 5))
    
    
    plt.subplot(1, 2, 1)
    plt.plot(history['train_acc'], label='Train')
    plt.plot(history['val_acc'], label='Validation')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
   
    plt.subplot(1, 2, 2)
    plt.plot(history['train_loss'], label='Train')
    plt.plot(history['val_loss'], label='Validation')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

# Main execution
def main():
    
    data_dir = '/kaggle/input/fish-split/Split'  
    
    
    
    train_dir = os.path.join(data_dir, 'train')
    val_dir = os.path.join(data_dir, 'val')
    test_dir = os.path.join(data_dir, 'test')
    
    print(f"Data directory: {data_dir}")
    print(f"Data directory exists: {os.path.exists(data_dir)}")
    print(f"Train directory: {train_dir}")
    print(f"Train directory exists: {os.path.exists(train_dir)}")
    
    
    for dir_path in [train_dir, val_dir, test_dir]:
        if not os.path.exists(dir_path):
            print(f"Warning: Directory not found: {dir_path}")
    
    
    train_transform, val_transform = get_transforms()
    
    
    train_dataset = CustomDataset(train_dir, transform=train_transform)
    val_dataset = CustomDataset(val_dir, transform=val_transform)
    test_dataset = CustomDataset(test_dir, transform=val_transform)
    
    
    class_names = train_dataset.classes
    print(f"Classes: {class_names}")
    
    
    batch_size = 16  
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    
    
    dataloaders = {
        'train': train_loader,
        'val': val_loader
    }
    
    
    model = create_vgg16_model(num_classes=len(class_names), fine_tune=True, dropout_rate=0.5)
    model = model.to(device)
    
    
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  
    
    
    optimizer = optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()), 
        lr=0.0001, 
        weight_decay=0.1  
    )
    
    # Set up learning rate scheduler 
    from torch.optim.lr_scheduler import OneCycleLR
    
    
    total_steps = len(train_loader) * 40  
    
    scheduler = OneCycleLR(
        optimizer,
        max_lr=0.001,  
        total_steps=total_steps,
        pct_start=0.1,  
        div_factor=25, 
        final_div_factor=1000,  
        anneal_strategy='cos'  
    )
    
    
    model, history = train_model(
        model=model,
        dataloaders=dataloaders,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        num_epochs=40,  
        patience=10,  
        mixup_alpha=0.3,  
        
    )
    
    
    plot_training_history(history)
    
    # Check for overfitting/underfitting
    fit_diagnosis = diagnose_model_fit(history)
    print(f"\nModel fit diagnosis: {fit_diagnosis['diagnosis']}")
    
    
    accuracy, cm = evaluate_model(model, test_loader, class_names)
    
    # Save the model
    torch.save(model.state_dict(), 'vgg16_fish_disease_model_improved.pth')
    print("Model saved")

# Call the main function
if __name__ == "__main__":
    main()


****VGG 16 parameter checking

In [ ]:


class CustomVGG16(nn.Module):
    def __init__(self, num_classes=7, dropout_rate=0.3):
        super(CustomVGG16, self).__init__()
        
        # Load pre-trained VGG16 model
        vgg16 = models.vgg16(pretrained=True)
        
        # Use the convolutional layers from VGG16
        self.features = vgg16.features
        
        # Create a custom classifier with regularization
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x

# Function to count parameters
def count_parameters(model):
    """Count the total and trainable parameters in the model"""
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

# Create the model
model = CustomVGG16(num_classes=7, dropout_rate=0.5)

# Count parameters
total_params, trainable_params = count_parameters(model)

# Get original VGG16 parameters for comparison
original_vgg16 = models.vgg16(pretrained=True)
original_total, _ = count_parameters(original_vgg16)


print(f"Your CustomVGG16 Model:")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print("\nBreakdown by layer:")

# Detailed parameter breakdown
print("\nFeatures (Convolutional layers):")
features_params = sum(p.numel() for p in model.features.parameters())
print(f"  Parameters: {features_params:,}")

print("\nClassifier (Fully connected layers):")
for i, layer in enumerate(model.classifier):
    if hasattr(layer, 'weight'):
        layer_params = sum(p.numel() for p in layer.parameters())
        print(f"  Layer {i} ({layer.__class__.__name__}): {layer_params:,} parameters")

print(f"\nOriginal VGG16 total parameters: {original_total:,}")
print(f"Difference: {total_params - original_total:,} parameters")



# ****Gradcam on VGG 16

In [ ]:

# Define your CustomVGG16 class 
class CustomVGG16(nn.Module):
    def __init__(self, num_classes=7, dropout_rate=0.3):
        super(CustomVGG16, self).__init__()
        
        # Load pre-trained VGG16 model
        vgg16 = models.vgg16(pretrained=True)
        
        # Use the convolutional layers from VGG16
        self.features = vgg16.features
        
        # Create a custom classifier with regularization
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x

# Define the GradCAM class
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        
        # Register hooks
        def forward_hook(module, input, output):
            self.activations = output.detach()
        
        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0].detach()
        
        # Register hooks
        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_backward_hook(backward_hook)
        
        # Set model to evaluation mode
        self.model.eval()
    
    def __call__(self, input_image, target_class=None):
        # Forward pass
        output = self.model(input_image)
        
        # Get the class index
        if target_class is None:
            pred = output.argmax(dim=1)[0]
            class_idx = pred.item()
        else:
            class_idx = target_class
        
        # Get the prediction probability
        pred_prob = F.softmax(output, dim=1)[0, class_idx].item()
        
        # Zero gradients
        self.model.zero_grad()
        
        # Backward pass with the target class
        target = output[0, class_idx]
        target.backward()
        
        # Get the gradients and activations
        gradients = self.gradients
        activations = self.activations
        
        # Global average pooling of gradients
        weights = torch.mean(gradients, dim=(2, 3), keepdim=True)
        
        # Weighted sum of activation maps
        cam = torch.sum(weights * activations, dim=1, keepdim=True)
        
        # ReLU to keep only positive contributions
        cam = F.relu(cam)
        
        # Normalize the CAM
        cam = F.interpolate(cam, size=input_image.shape[2:], mode='bilinear', align_corners=False)
        cam = cam - cam.min()
        cam = cam / (cam.max() + 1e-8)
        
        # Convert to numpy
        heatmap = cam[0, 0].cpu().numpy()
        
        return heatmap, class_idx, pred_prob

# Function to preprocess images
def preprocess_image(image_path, transform=None):
    # Load image
    img = Image.open(image_path).convert('RGB')
    original_img = np.array(img)
    
    # Apply transformations
    if transform is None:
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    preprocessed_img = transform(img).unsqueeze(0)
    
    return preprocessed_img, original_img

# Function to visualize Grad-CAM
def visualize_gradcam(original_img, heatmap, class_name, pred_prob, alpha=0.5):
    # Convert heatmap to RGB
    heatmap_rgb = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    heatmap_rgb = cv2.cvtColor(heatmap_rgb, cv2.COLOR_BGR2RGB)
    
    # Resize heatmap to match original image
    heatmap_rgb = cv2.resize(heatmap_rgb, (original_img.shape[1], original_img.shape[0]))
    
    # Overlay heatmap on original image
    superimposed_img = heatmap_rgb * alpha + original_img * (1 - alpha)
    superimposed_img = np.uint8(superimposed_img)
    
    # Create figure with original and overlaid images
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    
    # Original image
    ax[0].imshow(original_img)
    ax[0].set_title('Original Image')
    ax[0].axis('off')
    
    # Overlaid image
    ax[1].imshow(superimposed_img)
    ax[1].set_title(f'Grad-CAM: {class_name} ({pred_prob:.2%})')
    ax[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return superimposed_img

# Function to apply Grad-CAM to an image
def apply_gradcam(model, image_path, class_names, transform=None, target_class=None):
    # Preprocess image
    input_tensor, original_img = preprocess_image(image_path, transform)
    
    # Move to device
    device = next(model.parameters()).device
    input_tensor = input_tensor.to(device)
    
    # Get target layer (last convolutional layer in VGG16)
    target_layer = model.features[-1]
    
    # Initialize Grad-CAM
    grad_cam = GradCAM(model, target_layer)
    
    # Generate heatmap
    heatmap, class_idx, pred_prob = grad_cam(input_tensor, target_class)
    
    # Get class name
    class_name = class_names[class_idx] if class_idx < len(class_names) else f"Class {class_idx}"
    
    # Visualize
    superimposed_img = visualize_gradcam(original_img, heatmap, class_name, pred_prob)
    
    print(f"Predicted class: {class_name} with probability {pred_prob:.2%}")
    
    return superimposed_img, class_idx, pred_prob

# MAIN 

# 1. Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. Create model instance
model = CustomVGG16(num_classes=7)  # Adjust number of classes as needed

# 3. Load saved model weights

import glob
model_paths = glob.glob("/kaggle/input/notebooknewfish54601c88a6/vgg16_fish_disease_model_improved.pth", recursive=True)

if model_paths:
    model_path = model_paths[0]
    print(f"Found model at: {model_path}")
    
   
    try:
        model.load_state_dict(torch.load(model_path, map_location=device))
        print("Model loaded successfully!")
    except Exception as e:
        print(f"Error loading model: {e}")
else:
    print("Model file not found. Please upload it to Kaggle.")
# 4. Move model to device and set to evaluation mode
model = model.to(device)
model.eval()

# 5. Define class names 
class_names = ['Bacterial Red disease',
            'Bacterial diseases - Aeromoniasis',
            'Bacterial gill disease', 
            'Fungal diseases Saprolegniasis', 
            'Healthy Fish', 
            'Parasitic diseases', 
            'Viral diseases White tail disease' ]

# 6. Apply Grad-CAM to an image
image_path =  '/kaggle/input/fish-split/Split/val/Viral diseases White tail disease/Viral diseases White tail disease (21).jpg'
superimposed_img, class_idx, pred_prob = apply_gradcam(model, image_path, class_names)

# 7. Optional: Apply Grad-CAM to multiple images
def process_multiple_images(model, image_paths, class_names):
    results = []
    for img_path in image_paths:
        print(f"\nProcessing {img_path}...")
        result = apply_gradcam(model, img_path, class_names)
        results.append(result)
    return results



# ****Gradcam on Aquascan

In [ ]:

# Define the AquaScan-EfficientNet-Hybrid model class
class AquaScanEfficientNetHybrid(nn.Module):
    def __init__(self, num_classes=7, model_name='efficientnet_b0', use_pretrained=True):
        super(AquaScanEfficientNetHybrid, self).__init__()
        
        # Load pre-trained EfficientNet
        if model_name == 'efficientnet_b0':
            self.backbone = models.efficientnet_b0(weights='IMAGENET1K_V1' if use_pretrained else None)
        elif model_name == 'efficientnet_b1':
            self.backbone = models.efficientnet_b1(weights='IMAGENET1K_V1' if use_pretrained else None)
        elif model_name == 'efficientnet_b2':
            self.backbone = models.efficientnet_b2(weights='IMAGENET1K_V1' if use_pretrained else None)
        else:
            raise ValueError(f"Unsupported model name: {model_name}")
        
        # Extract features only 
        self.features = nn.Sequential(*list(self.backbone.children())[:-1])
        
        # Get the number of features from the backbone
        if model_name == 'efficientnet_b0':
            backbone_features = 1280
        elif model_name == 'efficientnet_b1':
            backbone_features = 1280
        elif model_name == 'efficientnet_b2':
            backbone_features = 1408
        
        
        self.activation = nn.LeakyReLU(0.1, inplace=True)
        
        # Original AquaScan convolutional blocks with residual connections
        # Block 1 with residual connection
        self.conv_block1_main = nn.Sequential(
            nn.Conv2d(backbone_features, backbone_features, kernel_size=3, padding=1),
            nn.BatchNorm2d(backbone_features),
            self.activation,
            nn.Conv2d(backbone_features, backbone_features, kernel_size=3, padding=1),
            nn.BatchNorm2d(backbone_features),
            nn.Dropout(0.1)
        )
        
        # Block 2 with residual connection
        self.conv_block2_main = nn.Sequential(
            nn.Conv2d(backbone_features, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            self.activation,
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.Dropout(0.1)
        )
        self.conv_block2_shortcut = nn.Sequential(
            nn.Conv2d(backbone_features, 512, kernel_size=1, stride=1),
            nn.BatchNorm2d(512)
        )
        
        # Global Average Pooling
        self.gap = nn.AdaptiveAvgPool2d(1)
        
        # Fully Connected Blocks with more capacity
        self.fc_block = nn.Sequential(
            nn.Linear(512, 1024),
            self.activation,
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            self.activation,
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            self.activation,
            nn.Dropout(0.3)
        )
        
        # Output Layer
        self.output_layer = nn.Linear(256, num_classes)
    
    def forward(self, x):
        # Get features from EfficientNet backbone
        x = self.features(x)
        
        # Block 1 with residual connection
        identity = x
        x = self.conv_block1_main(x)
        x = x + identity  # Residual connection
        x = self.activation(x)
        
        # Block 2 with residual connection
        identity = self.conv_block2_shortcut(x)
        x = self.conv_block2_main(x)
        x = x + identity  # Residual connection
        x = self.activation(x)
        
        # Global Average Pooling
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = self.fc_block(x)
        x = self.output_layer(x)
        
        return x

# Enhanced GradCAM class with better visualization
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        
        # Register hooks
        def forward_hook(module, input, output):
            self.activations = output.detach()
        
        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0].detach()
        
        # Register hooks
        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_backward_hook(backward_hook)
        
        # Set model to evaluation mode
        self.model.eval()
    
    def __call__(self, input_image, target_class=None):
        # Forward pass
        output = self.model(input_image)
        
        # Get the class index
        if target_class is None:
            pred = output.argmax(dim=1)[0]
            class_idx = pred.item()
        else:
            class_idx = target_class
        
        # Get the prediction probability
        pred_prob = F.softmax(output, dim=1)[0, class_idx].item()
        
        # Zero gradients
        self.model.zero_grad()
        
        # Backward pass with the target class
        target = output[0, class_idx]
        target.backward()
        
        # Get the gradients and activations
        gradients = self.gradients
        activations = self.activations
        
        # Global average pooling of gradients
        weights = torch.mean(gradients, dim=(2, 3), keepdim=True)
        
        # Weighted sum of activation maps
        cam = torch.sum(weights * activations, dim=1, keepdim=True)
        
        # ReLU to keep only positive contributions
        cam = F.relu(cam)
        
        # Normalize the CAM
        cam = F.interpolate(cam, size=input_image.shape[2:], mode='bilinear', align_corners=False)
        cam = cam - cam.min()
        cam = cam / (cam.max() + 1e-8)
        
        # Convert to numpy
        heatmap = cam[0, 0].cpu().numpy()
        
        return heatmap, class_idx, pred_prob

# Function to preprocess images
def preprocess_image(image_path, transform=None):
    try:
        # Load image
        img = Image.open(image_path).convert('RGB')
        original_img = np.array(img)
        
        # Apply transformations
        if transform is None:
            transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        
        preprocessed_img = transform(img).unsqueeze(0)
        
        return preprocessed_img, original_img
    except Exception as e:
        print(f"Error preprocessing image {image_path}: {e}")
        return None, None

# Enhanced visualization function with better contrast and multiple views
def visualize_gradcam(original_img, heatmap, class_name, pred_prob, alpha=0.7, save_path=None):
    
    heatmap_eq = cv2.equalizeHist(np.uint8(255 * heatmap)) / 255.0
    
    # Convert heatmap to RGB with JET colormap (red = high activation)
    heatmap_rgb = cv2.applyColorMap(np.uint8(255 * heatmap_eq), cv2.COLORMAP_JET)
    heatmap_rgb = cv2.cvtColor(heatmap_rgb, cv2.COLOR_BGR2RGB)
    
    # Resize heatmap to match original image
    heatmap_rgb = cv2.resize(heatmap_rgb, (original_img.shape[1], original_img.shape[0]))
    
    # Overlay heatmap on original image with higher alpha for better visibility
    superimposed_img = heatmap_rgb * alpha + original_img * (1 - alpha)
    superimposed_img = np.clip(superimposed_img, 0, 255).astype(np.uint8)
    
    # Create a more comprehensive visualization with 4 panels
    fig, ax = plt.subplots(2, 2, figsize=(15, 12))
    
    # Original image
    ax[0, 0].imshow(original_img)
    ax[0, 0].set_title('Original Image')
    ax[0, 0].axis('off')
    
    # Heatmap only
    ax[0, 1].imshow(heatmap_eq, cmap='jet')
    ax[0, 1].set_title('GradCAM Heatmap')
    ax[0, 1].axis('off')
    
    # Overlaid image with high transparency
    overlay_light = heatmap_rgb * 0.3 + original_img * 0.7
    overlay_light = np.clip(overlay_light, 0, 255).astype(np.uint8)
    ax[1, 0].imshow(overlay_light)
    ax[1, 0].set_title('Subtle Overlay (30%)')
    ax[1, 0].axis('off')
    
    # Overlaid image with stronger effect
    ax[1, 1].imshow(superimposed_img)
    ax[1, 1].set_title(f'Strong Overlay (70%): {class_name} ({pred_prob:.2%})')
    ax[1, 1].axis('off')
    
    plt.tight_layout()
    
    # Save the visualization if a path is provided
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()
    
    return superimposed_img

# Function to get available target layers from the model
def get_target_layers(model):
    target_layers = {
        "backbone_last": model.features[-1],
        "conv_block1_first": model.conv_block1_main[0],
        "conv_block1_last": model.conv_block1_main[3],
        "conv_block2_first": model.conv_block2_main[0],
        "conv_block2_last": model.conv_block2_main[3]
    }
    return target_layers

# Enhanced function to apply GradCAM to an image with better target layer selection
def apply_gradcam(model, image_path, class_names, target_layer_name=None, transform=None, target_class=None, save_path=None):
    # Preprocess image
    input_tensor, original_img = preprocess_image(image_path, transform)
    
    if input_tensor is None:
        print(f"Failed to load image: {image_path}")
        return None, None, None
    
    # Move to device
    device = next(model.parameters()).device
    input_tensor = input_tensor.to(device)
    
    # Get available target layers
    target_layers = get_target_layers(model)
    
    # Select target layer
    if target_layer_name is None or target_layer_name not in target_layers:
        print(f"Target layer '{target_layer_name}' not found. Using conv_block2_last.")
        target_layer = target_layers["conv_block2_last"]
        target_layer_name = "conv_block2_last"
    else:
        target_layer = target_layers[target_layer_name]
    
    print(f"Using target layer: {target_layer_name}")
    
    # Initialize Grad-CAM
    grad_cam = GradCAM(model, target_layer)
    
    # Generate heatmap
    heatmap, class_idx, pred_prob = grad_cam(input_tensor, target_class)
    
    # Get class name
    class_name = class_names[class_idx] if class_idx < len(class_names) else f"Class {class_idx}"
    
    # Create save path if needed
    if save_path is None and target_layer_name is not None:
        base_name = os.path.basename(image_path)
        save_dir = "gradcam_results"
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"{os.path.splitext(base_name)[0]}_{target_layer_name}.png")
    
    # Visualize
    superimposed_img = visualize_gradcam(original_img, heatmap, class_name, pred_prob, save_path=save_path)
    
    print(f"Predicted class: {class_name} with probability {pred_prob:.2%}")
    
    return superimposed_img, class_idx, pred_prob

# Function to compare GradCAM results from different layers
def compare_layers(model, image_path, class_names, transform=None, target_class=None):
    # Get available target layers
    target_layers = get_target_layers(model)
    
    # Create a directory for saving results
    save_dir = "gradcam_layer_comparison"
    os.makedirs(save_dir, exist_ok=True)
    
    # Process each layer
    results = {}
    for layer_name, layer in target_layers.items():
        print(f"\nProcessing layer: {layer_name}")
        save_path = os.path.join(save_dir, f"{os.path.basename(image_path).split('.')[0]}_{layer_name}.png")
        result = apply_gradcam(
            model=model,
            image_path=image_path,
            class_names=class_names,
            target_layer_name=layer_name,
            transform=transform,
            target_class=target_class,
            save_path=save_path
        )
        results[layer_name] = result
    
    return results

# Function to process multiple images
def process_multiple_images(model, image_paths, class_names, target_layer_name=None):
    results = []
    for img_path in image_paths:
        print(f"\nProcessing {img_path}...")
        result = apply_gradcam(model, img_path, class_names, target_layer_name)
        results.append(result)
    return results

# Function to process all images in a directory
def process_directory(model, directory_path, class_names, target_layer_name=None, pattern="*.jpg"):
    # Find all images in the directory
    image_paths = []
    for ext in ["*.jpg", "*.jpeg", "*.png", "*.bmp"]:
        image_paths.extend(glob.glob(os.path.join(directory_path, ext)))
        image_paths.extend(glob.glob(os.path.join(directory_path, "**", ext), recursive=True))
    
    if not image_paths:
        print(f"No images found in {directory_path}")
        return []
    
    print(f"Found {len(image_paths)} images to process")
    return process_multiple_images(model, image_paths, class_names, target_layer_name)

# MAIN 
if __name__ == "__main__":
    try:
        #  Setting device
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")

        #  Creating model instance
        model = AquaScanEfficientNetHybrid(num_classes=7)
        
        #  Saving model 
        model_path = "/kaggle/input/aquascanfishdisease/pytorch/default/1/aquascan_model_20250412_101409.pth"
        try:
            model.load_state_dict(torch.load(model_path, map_location=device))
            print(f"Model loaded successfully from {model_path}")
        except Exception as e:
            print(f"Error loading model: {e}")
            raise
            
        model = model.to(device)
        model.eval()

        # 4. The classes
        class_names = [
            'Bacterial Red disease',
            'Bacterial diseases - Aeromoniasis',
            'Bacterial gill disease', 
            'Fungal diseases Saprolegniasis', 
            'Healthy Fish', 
            'Parasitic diseases', 
            'Viral diseases White tail disease' 
            
        ]

        # 5. Appling GradCAM to an image with different layers for comparison
        image_path = '/kaggle/input/fish-split/Split/val/Bacterial Red disease/Bacterial Red disease (156).jpg'
        
        # Comparing different layers to find which one gives the best visualization
        print("\nComparing different layers for GradCAM visualization...")
        compare_layers(model, image_path, class_names)
        
        
        print("\nGradCAM implementation for AquaScan-EfficientNet-Hybrid completed!")
        
    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()

# ****Download files from working folder

In [ ]:
import shutil

# Specify the folder you want to download
folder_to_download = '/kaggle/working/val'  # Replace with the name of your folder
output_zip = 'val.zip'

# Compress the folder into a zip file
shutil.make_archive(output_zip.replace('.zip', ''), 'zip', folder_to_download)

# If running on Kaggle, use this to download the zip file
from IPython.display import FileLink
FileLink(output_zip)
